In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Итоговое задание семинара (3.3.Final)

### Код из библиотеки dlnlputils репозитория https://github.com/Samsung-IT-Academy/stepik-dl-nlp

In [4]:
#########stepik-dl-nlp/dlnlputils/data/base.py#########

import collections
import re

import numpy as np

TOKEN_RE = re.compile(r'[\w\d]+')


def tokenize_text_simple_regex(txt, min_token_size=4):
    txt = txt.lower()
    all_tokens = TOKEN_RE.findall(txt)
    return [token for token in all_tokens if len(token) >= min_token_size]


def tokenize_corpus(texts, tokenizer=tokenize_text_simple_regex, **tokenizer_kwargs):
    return [tokenizer(text, **tokenizer_kwargs) for text in texts]

def tokenize_corpus_verbose(texts, tokenizer=tokenize_text_simple_regex, verbose_chunk=1000, **tokenizer_kwargs):
    tokenize_texts = []
    for i, text in enumerate(texts):
        tokenize_texts.append(tokenizer(text, **tokenizer_kwargs))
        if i % verbose_chunk == 0:
            print('Complete: {}/{}'.format(i,len(texts)))
    return tokenize_texts

def texts_to_token_ids(tokenized_texts, word2id):
    return [[word2id[token] for token in text if token in word2id]
            for text in tokenized_texts]


def build_vocabulary(tokenized_texts, max_size=1000000, max_doc_freq=0.8, min_count=5, pad_word=None):
    word_counts = collections.defaultdict(int)
    doc_n = 0

    # посчитать количество документов, в которых употребляется каждое слово
    # а также общее количество документов
    for txt in tokenized_texts:
        doc_n += 1
        unique_text_tokens = set(txt)
        for token in unique_text_tokens:
            word_counts[token] += 1

    # убрать слишком редкие и слишком частые слова
    word_counts = {word: cnt for word, cnt in word_counts.items()
                   if cnt >= min_count and cnt / doc_n <= max_doc_freq}

    # отсортировать слова по убыванию частоты
    sorted_word_counts = sorted(word_counts.items(),
                                reverse=True,
                                key=lambda pair: pair[1])

    # добавим несуществующее слово с индексом 0 для удобства пакетной обработки
    if pad_word is not None:
        sorted_word_counts = [(pad_word, 0)] + sorted_word_counts

    # если у нас по прежнему слишком много слов, оставить только max_size самых частотных
    if len(word_counts) > max_size:
        sorted_word_counts = sorted_word_counts[:max_size]

    # нумеруем слова
    word2id = {word: i for i, (word, _) in enumerate(sorted_word_counts)}

    # нормируем частоты слов
    word2freq = np.array([cnt / doc_n for _, cnt in sorted_word_counts], dtype='float32')

    return word2id, word2freq



#########stepik-dl-nlp/dlnlputils/data/bag_of_words.py#########

import numpy as np
import scipy.sparse
import torch
from torch.utils.data import Dataset


def vectorize_texts(tokenized_texts, word2id, word2freq, mode='tfidf', scale=True):
    #modified by me 
    #add 'lftidf', 'tflidf', 'ltflidf', 'ltf', 'lidf'
    
    assert mode in {'tfidf', 'idf', 'tf', 'bin', 'ltfidf', 'tflidf', 'tflidf_v2', 'ltf', 'tfpmi'}

    # считаем количество употреблений каждого слова в каждом документе
    result = scipy.sparse.dok_matrix((len(tokenized_texts), len(word2id)), dtype='float32')
    for text_i, text in enumerate(tokenized_texts):
        for token in text:
            if token in word2id:
                result[text_i, word2id[token]] += 1

    # получаем бинарные вектора "встречается или нет"
    if mode == 'bin':
        result = (result > 0).astype('float32')

    # получаем вектора относительных частот слова в документе
    elif mode == 'tf':
        result = result.tocsr()
        result = result.multiply(1 / result.sum(1))

    # полностью убираем информацию о количестве употреблений слова в данном документе,
    # но оставляем информацию о частотности слова в корпусе в целом
    elif mode == 'idf':
        result = (result > 0).astype('float32').multiply(1 / word2freq)

    # учитываем всю информацию, которая у нас есть:
    # частоту слова в документе и частоту слова в корпусе
    elif mode == 'tfidf':
        result = result.tocsr()
        result = result.multiply(1 / result.sum(1))  # разделить каждую строку на её длину
        result = result.multiply(1 / word2freq)  # разделить каждый столбец на вес слова

    elif mode == 'ltf': # lTF=ln⁡(TF+1)
        result = result.tocsr()
        result = result.multiply(1 / result.sum(1))
        result = scipy.sparse.dok_matrix(np.log(result.toarray()+1))
 
    elif mode == 'lidf': # lIDF=ln⁡(n/IDF+1)
        result = (result > 0).astype('float32').multiply(len(tokenized_texts) / word2freq)
        result = scipy.sparse.dok_matrix(np.log(result.toarray()+1))

        
    elif mode == 'ltfidf': # lTFIDF=ln⁡(TF+1)⋅IDF
        result = result.tocsr() #переводим матрицу в режим быстрой работы со строками (это очень важно!!)
        result = result.multiply(1/result.sum(1)) # разделить каждую строку на её длину
        result = scipy.sparse.dok_matrix(np.log(result.toarray()+1))
        result = result.multiply(1 / word2freq) # разделить каждый столбец на вес слова
        

    elif mode == 'tflidf': # lTFIDF=TF⋅ln⁡(1/IDF+1)
        result = result.tocsr() #переводим матрицу в режим быстрой работы со строками (это очень важно!!)
        result = result.multiply(1/result.sum(1)) # разделить каждую строку на её длину
        result = result.multiply(np.log(1 / word2freq + 1)) # разделить каждый столбец на вес слова

    elif mode == 'tflidf_v2': # lTFIDF=TF⋅ln⁡(n/IDF+1)
        result = result.tocsr() #переводим матрицу в режим быстрой работы со строками (это очень важно!!)
        result = result.multiply(1/result.sum(1)) # разделить каждую строку на её длину
        result = result.multiply(np.log(len(tokenized_texts) / word2freq + 1)) # разделить каждый столбец на вес слова
        
    elif mode == 'tfpmi': # TFPMI=TF⋅PMI
        result = result.tocsr()
        result = result.multiply(1 / result.sum(1))  # разделить каждую строку на её длину
        result = result.multiply(word2freq)  # домножить каждую строку на word2freq (это массив PMI Scores)

    if scale:
        result = result.tocsc()
        result -= result.min()
        result /= (result.max() + 1e-6)

    return result.tocsr()


class SparseFeaturesDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, idx):
        cur_features = torch.from_numpy(self.features[idx].toarray()[0]).float()
        cur_label = torch.from_numpy(np.asarray(self.targets[idx])).long()
        return cur_features, cur_label
    
    
#########stepik-dl-nlp/dlnlputils/pipeline.py#########

import copy
import datetime
import random
import traceback

import numpy as np
import torch
from torch.utils.data import DataLoader


def init_random_seed(value=0):
    random.seed(value)
    np.random.seed(value)
    torch.manual_seed(value)
    torch.cuda.manual_seed(value)
    torch.backends.cudnn.deterministic = True


def copy_data_to_device(data, device):
    if torch.is_tensor(data):
        return data.to(device)
    elif isinstance(data, (list, tuple)):
        return [copy_data_to_device(elem, device) for elem in data]
    raise ValueError('Недопустимый тип данных {}'.format(type(data)))


def print_grad_stats(model):
    mean = 0
    std = 0
    norm = 1e-5
    for param in model.parameters():
        grad = getattr(param, 'grad', None)
        if grad is not None:
            mean += grad.data.abs().mean()
            std += grad.data.std()
            norm += 1
    mean /= norm
    std /= norm
    print(f'Mean grad {mean}, std {std}, n {norm}')


def train_eval_loop(model, train_dataset, val_dataset, criterion,
                    lr=1e-4, epoch_n=10, batch_size=32,
                    device=None, early_stopping_patience=10, l2_reg_alpha=0,
                    max_batches_per_epoch_train=10000,
                    max_batches_per_epoch_val=1000,
                    data_loader_ctor=DataLoader,
                    optimizer_ctor=None,
                    lr_scheduler_ctor=None,
                    shuffle_train=True,
                    dataloader_workers_n=0,
                    best_acc_type = 'loss',
                    test_dataset = None,
                    experiment_name = 'NoName',
                    no_calculate_accuracy = False):
    """
    v2.1
    Цикл для обучения модели. После каждой эпохи качество модели оценивается по отложенной выборке.
    :param model: torch.nn.Module - обучаемая модель
    :param train_dataset: torch.utils.data.Dataset - данные для обучения
    :param val_dataset: torch.utils.data.Dataset - данные для оценки качества
    :param criterion: функция потерь для настройки модели
    :param lr: скорость обучения
    :param epoch_n: максимальное количество эпох
    :param batch_size: количество примеров, обрабатываемых моделью за одну итерацию
    :param device: cuda/cpu - устройство, на котором выполнять вычисления
    :param early_stopping_patience: наибольшее количество эпох, в течение которых допускается
        отсутствие улучшения модели, чтобы обучение продолжалось.
    :param l2_reg_alpha: коэффициент L2-регуляризации
    :param max_batches_per_epoch_train: максимальное количество итераций на одну эпоху обучения
    :param max_batches_per_epoch_val: максимальное количество итераций на одну эпоху валидации
    :param data_loader_ctor: функция для создания объекта, преобразующего датасет в батчи
        (по умолчанию torch.utils.data.DataLoader)
    :return: кортеж из двух элементов:
        - среднее значение функции потерь на валидации на лучшей эпохе
        - лучшая модель
    """
    
    '''
    modified by wisoffe
    best_acc_type: 'loss' or 'acc'
    experiment_name: 
    '''
    assert best_acc_type in {'loss', 'acc'}
    
    train_start_time = datetime.datetime.now()
    print("############## Start experiment with name: {} ##############".format(experiment_name))
    
    #statistics history
    history = {'acc': {'train': [0.0],
                       'val': [0.0]},
               'loss': {'train': [float('inf')],
                       'val': [float('inf')]}}
    
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    device = torch.device(device)
    model.to(device)

    if optimizer_ctor is None:
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2_reg_alpha)
    else:
        optimizer = optimizer_ctor(model.parameters(), lr=lr)

    if lr_scheduler_ctor is not None:
        lr_scheduler = lr_scheduler_ctor(optimizer)
    else:
        lr_scheduler = None

    train_dataloader = data_loader_ctor(train_dataset, batch_size=batch_size, shuffle=shuffle_train,
                                        num_workers=dataloader_workers_n)
    val_dataloader = data_loader_ctor(val_dataset, batch_size=batch_size, shuffle=False,
                                      num_workers=dataloader_workers_n)
    
    if best_acc_type == 'loss': #отбираем модель по минимальному loss
        best_val_metric = float('inf')
    elif best_acc_type == 'acc': #отбираем модель по максимальному accuracy
        best_val_metric = float('-inf')
        
    best_epoch_i = 0
    best_model = copy.deepcopy(model)
    
    
    for epoch_i in range(1, epoch_n + 1):
        try:
            #####train phase######
            epoch_start = datetime.datetime.now()
            train_accuracy_epoch = [] #for statistics
            train_loss_epoch = [] #for statistics
            
            model.train()
            
            for batch_i, (batch_x, batch_y) in enumerate(train_dataloader):
                if batch_i > max_batches_per_epoch_train:
                    print('Threshold max_batches_per_epoch_train exceeded!')
                    break

                batch_x = copy_data_to_device(batch_x, device)
                batch_y = copy_data_to_device(batch_y, device)

                pred = model(batch_x)
                loss = criterion(pred, batch_y)

                model.zero_grad()
                loss.backward()

                optimizer.step()

                train_loss_epoch.append(float(loss))
                
                if not no_calculate_accuracy:
                    train_accuracy_epoch.append(float((pred.argmax(dim=1) == batch_y.data).float().mean().data))
                    #train_accuracy_epoch.append(float((pred.detach().cpu().numpy().argmax(-1) == batch_y.detach().cpu().numpy()).mean()))
                else: train_accuracy_epoch.append(0.)
                    
            
            #####validation phase######
            model.eval()

            val_accuracy_epoch = [] #for statistics
            val_loss_epoch = [] #for statistics

            with torch.no_grad():
                for batch_i, (batch_x, batch_y) in enumerate(val_dataloader):
                    if batch_i > max_batches_per_epoch_val:
                        print('Threshold max_batches_per_epoch_val exceeded!')
                        break

                    batch_x = copy_data_to_device(batch_x, device)
                    batch_y = copy_data_to_device(batch_y, device)

                    pred = model(batch_x)
                    loss = criterion(pred, batch_y)
                    
                    if not no_calculate_accuracy:
                        val_accuracy_epoch.append(float((pred.argmax(dim=1) == batch_y.data).float().mean().data))
                        #val_accuracy_epoch.append(float((pred.detach().cpu().numpy().argmax(-1) == batch_y.detach().cpu().numpy()).mean()))
                    else:
                        val_accuracy_epoch.append(0.)
                    val_loss_epoch.append(float(loss))

            
            ########ending of epoch#########
            
            history['acc']['train'].append(sum(train_accuracy_epoch) / len(train_accuracy_epoch))
            history['loss']['train'].append(sum(train_loss_epoch) / len(train_loss_epoch))  

            history['acc']['val'].append(sum(val_accuracy_epoch) / len(val_accuracy_epoch))
            history['loss']['val'].append(sum(val_loss_epoch) / len(val_loss_epoch))
            
            
            #save best model
            best_model_saved = False
            if (best_acc_type == 'loss' and history['loss']['val'][-1] < best_val_metric) or \
                    (best_acc_type == 'acc' and history['acc']['val'][-1] > best_val_metric):
                #отбираем модель по минимальному loss или максимальному accuracy
                best_epoch_i = epoch_i
                best_val_metric = history[best_acc_type]['val'][-1]
                best_model = copy.deepcopy(model)
                best_model_saved = True
            #check for break training
            elif epoch_i - best_epoch_i > early_stopping_patience:
                print('Модель не улучшилась за последние {} эпох, прекращаем обучение'.format(
                    early_stopping_patience))
                break

            if lr_scheduler is not None:
                lr_scheduler.step(history['loss']['val'][-1])
            
            #output statistics
            
            print('Epoch = {:>3},   ACC: val = {:.3f}, train = {:.3f}    LOSS: val = {:.3f}, train = {:.3f}   SAVE: {}, Time: {:0.2f}s'\
                  .format(epoch_i,
                          history['acc']['val'][-1], 
                          history['acc']['train'][-1],
                          history['loss']['val'][-1],
                          history['loss']['train'][-1],
                          best_model_saved,
                          (datetime.datetime.now() - epoch_start).total_seconds()),
                  flush=True)

        except KeyboardInterrupt:
            print('Досрочно остановлено пользователем')
            break
        except Exception as ex:
            print('Ошибка при обучении: {}\n{}'.format(ex, traceback.format_exc()))
            break
            
    print(' ')
    print("BEST MODEL: ACC: val = {:.3f}, train = {:.3f}, LOSS: val = {:.3f}, train = {:.3f}, on epoch = {}, metric type = {}, Full train time = {:0.2f}s"\
                  .format(history['acc']['val'][best_epoch_i], 
                          history['acc']['train'][best_epoch_i],
                          history['loss']['val'][best_epoch_i],
                          history['loss']['train'][best_epoch_i],
                          best_epoch_i,
                          best_acc_type,
                          (datetime.datetime.now() - train_start_time).total_seconds()))
    print("************** End experiment with name: {} **************".format(experiment_name))
    print(' ')
    history['BEST'] = {}
    history['BEST']['epoch'] = best_epoch_i
    history['BEST']['dict_size'] = batch_x.shape[-1]
    
    
    #calculate and save final metrics best_model on train/val/test datasets
    if test_dataset is not None:
        history['BEST']['acc'] = {}
        history['BEST']['loss'] = {}
        
        #save validation metrics (no calculate again)
        history['BEST']['acc']['val'] = history['acc']['val'][best_epoch_i]
        history['BEST']['loss']['val'] = history['loss']['val'][best_epoch_i]
        
        #calculate and save train metrics
        train_pred = predict_with_model(best_model, train_dataset, return_labels=True)
        history['BEST']['loss']['train'] = float(F.cross_entropy(torch.from_numpy(train_pred[0]),
                             torch.from_numpy(train_pred[1]).long()))
        history['BEST']['acc']['train'] = accuracy_score(train_pred[1], train_pred[0].argmax(-1))
        
        #calculate and save test metrics
        test_pred = predict_with_model(best_model, test_dataset, return_labels=True)
        history['BEST']['loss']['test'] = float(F.cross_entropy(torch.from_numpy(test_pred[0]),
                             torch.from_numpy(test_pred[1]).long()))
        history['BEST']['acc']['test'] = accuracy_score(test_pred[1], test_pred[0].argmax(-1))    
    
    
    return history, best_model


def predict_with_model(model, dataset, device=None, batch_size=32, num_workers=0, return_labels=False):
    """
    :param model: torch.nn.Module - обученная модель
    :param dataset: torch.utils.data.Dataset - данные для применения модели
    :param device: cuda/cpu - устройство, на котором выполнять вычисления
    :param batch_size: количество примеров, обрабатываемых моделью за одну итерацию
    :return: numpy.array размерности len(dataset) x *
    """
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    results_by_batch = []

    device = torch.device(device)
    model.to(device)
    model.eval()

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    labels = []
    with torch.no_grad():
        import tqdm
        for batch_x, batch_y in tqdm.tqdm(dataloader, total=len(dataset)/batch_size):
            batch_x = copy_data_to_device(batch_x, device)

            if return_labels:
                labels.append(batch_y.numpy())

            batch_pred = model(batch_x)
            results_by_batch.append(batch_pred.detach().cpu().numpy())

    if return_labels:
        return np.concatenate(results_by_batch, 0), np.concatenate(labels, 0)
    else:
        return np.concatenate(results_by_batch, 0)


#########stepik-dl-nlp/dlnlputils/nnets.py#########

from torch.utils.data import Dataset


def ensure_length(txt, out_len, pad_value):
    if len(txt) < out_len:
        txt = list(txt) + [pad_value] * (out_len - len(txt))
    else:
        txt = txt[:out_len]
    return txt


class PaddedSequenceDataset(Dataset):
    def __init__(self, texts, targets, out_len=100, pad_value=0):
        self.texts = texts
        self.targets = targets
        self.out_len = out_len
        self.pad_value = pad_value

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        txt = self.texts[item]

        txt = ensure_length(txt, self.out_len, self.pad_value)
        txt = torch.tensor(txt, dtype=torch.long)

        target = torch.tensor(self.targets[item], dtype=torch.long)

        return txt, target

#########stepik-dl-nlp/dlnlputils/embeddings.py#########

class Embeddings:
    def __init__(self, embeddings, word2id):
        self.embeddings = embeddings
        self.embeddings /= (np.linalg.norm(self.embeddings, ord=2, axis=-1, keepdims=True) + 1e-4)
        self.word2id = word2id
        self.id2word = {i: w for w, i in word2id.items()}

    def most_similar(self, positive=None, negative=None, topk=10, with_mean = False):
        #modified by wis, converted to gensim syntax
        
        if positive is not None:
            if type(positive) != list:
                positive = [positive]
            pos_vec = [self.get_vector(word) for word in positive]
            pos_len = len(positive)
        else:
            pos_vec = 0
            pos_len = 1
            
        if negative is not None:
            if type(negative) != list:
                negative = [negative]
            neg_vec = [self.get_vector(word) for word in negative]
            neg_len = len(negative)
        else:
            neg_vec = 0
            neg_len = 1
        
        if with_mean:
            result_vec = np.array(pos_vec).sum(0) / pos_len - np.array(neg_vec).sum(0) / neg_len
        else:
            result_vec = np.array(pos_vec).sum(0) - np.array(neg_vec).sum(0)
        
        return self.most_similar_by_vector(result_vec, topk=topk)
    
    def most_similar_legacy(self, word, topk=10):
        return self.most_similar_by_vector(self.get_vector(word), topk=topk)

    def analogy(self, a1, b1, a2, topk=10):
        a1_v = self.get_vector(a1)
        b1_v = self.get_vector(b1)
        a2_v = self.get_vector(a2)
        query = b1_v - a1_v + a2_v
        return self.most_similar_by_vector(query, topk=topk)

    def most_similar_by_vector(self, query_vector, topk=10):
        similarities = (self.embeddings * query_vector).sum(-1)
        best_indices = np.argpartition(-similarities, topk, axis=0)[:topk]
        result = [(self.id2word[i], similarities[i]) for i in best_indices]
        result.sort(key=lambda pair: -pair[1])
        return result

    def get_vector(self, word):
        if word not in self.word2id:
            raise ValueError('Неизвестное слово "{}"'.format(word))
        return self.embeddings[self.word2id[word]]

    def get_vectors(self, *words):
        word_ids = [self.word2id[i] for i in words]
        vectors = np.stack([self.embeddings[i] for i in word_ids], axis=0)
        return vectors

#########stepik-dl-nlp/dlnlputils/visualization.py#########

from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE


def plot_vectors(vectors, labels, how='tsne', ax=None, xy_lim=None):
    if how == 'tsne':
        projections = TSNE().fit_transform(vectors)
    elif how == 'svd':
        projections = TruncatedSVD().fit_transform(vectors)

    x = projections[:, 0]
    y = projections[:, 1]
    if xy_lim is not None:
        ax.set_xlim(xy_lim)
        ax.set_ylim(xy_lim)
    ax.scatter(x, y)
    for cur_x, cur_y, cur_label in zip(x, y, labels):
        ax.annotate(cur_label, (cur_x, cur_y))

### Мои наработки

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import spacy
!python -m spacy download ru_core_news_md
spacy_nlp = spacy.load('ru_core_news_md', disable=['parser', 'ner'])

def tokenize_text_spacy_lemmatize(txt, spacy_nlp, min_token_size=4, with_pos = True, remove_stopwords = False):
    doc = spacy_nlp(txt)
    
    if remove_stopwords:
        lemmatized_doc = [token for token in doc if (len(token) >= min_token_size) and (not token.is_stop)]
    else:
        lemmatized_doc = [token for token in doc if len(token) >= min_token_size]
    
    if with_pos:
        return ['_'.join([token.lemma_, token.pos_]) for token in lemmatized_doc]
    else:
        return [token.lemma_ for token in lemmatized_doc]

def tokenize_corpus_convert(tokenized_corpus, converter, addition = False):
    '''
    Convert each token in tokenized_corpus by converter
    
    Sample (PorterStemmer):
    import nltk
    ps = nltk.stemmer.PorterStemmer()
    tokenized_stemmed_corpus = tokenize_corpus_convert(tokenized_corpus, converter=ps.stem)
    
    Sample (SnowballStemmer):
    import nltk
    sno = nltk.stem.SnowballStemmer('english')
    tokenized_stemmed_corpus = tokenize_corpus_convert(tokenized_corpus, converter=sno.stem)
    
    Sample (WordNetLemmatizer):
    import nltk
    lemma = nltk.wordnet.WordNetLemmatizer()
    tokenized_lemmas_corpus = tokenize_corpus_convert(tokenized_corpus, converter=lemma.lemmatize)
    '''
    output = []
    if not addition: #возвращаем только преобразованные токены
        for doc in tokenized_corpus:
            output.append([converter(token) for token in doc])
    else: #возвращаем списк из исходных токенов, дополненных списком преобразованных
        for doc in tokenized_corpus:
            output.append(doc + [converter(token) for token in doc])        
    return output

def show_experiments_stats(histories, figsize = (16.0, 6.0), show_plots = True, only_BEST_MODEL_CALC = False):
    matplotlib.rcParams['figure.figsize'] = figsize
    
    for experiment_id in histories.keys():
        print('{:-<100}'.format(experiment_id))
        
        if not only_BEST_MODEL_CALC:
            epoch_max_acc = np.array(histories[experiment_id]['acc']['val']).argmax()
            print('Max val acc on:    Epoch = {:>3},   ACCURACY: val  = {:.3f}, train = {:.3f},   LOSS: val  = {:.3f}, train = {:.3f}'\
                  .format(epoch_max_acc, 
                          histories[experiment_id]['acc']['val'][epoch_max_acc], 
                          histories[experiment_id]['acc']['train'][epoch_max_acc],
                          histories[experiment_id]['loss']['val'][epoch_max_acc],
                          histories[experiment_id]['loss']['train'][epoch_max_acc]))
            epoch_min_loss = np.array(histories[experiment_id]['loss']['val']).argmin()
            print('Min val loss on:   Epoch = {:>3},   ACCURACY: val  = {:.3f}, train = {:.3f},   LOSS: val  = {:.3f}, train = {:.3f}'\
                  .format(epoch_min_loss, 
                          histories[experiment_id]['acc']['val'][epoch_min_loss], 
                          histories[experiment_id]['acc']['train'][epoch_min_loss],
                          histories[experiment_id]['loss']['val'][epoch_min_loss],
                          histories[experiment_id]['loss']['train'][epoch_min_loss]))
        
        if 'acc' in histories[experiment_id]['BEST']:
            print("BEST MODEL CALC:   Epoch = {:>3},   ACCURACY: test = {:.3f}, train = {:.3f},   LOSS: test = {:.3f}, train = {:.3f}  DICT SIZE = {}"\
                  .format(histories[experiment_id]['BEST']['epoch'], 
                          histories[experiment_id]['BEST']['acc']['test'],
                          histories[experiment_id]['BEST']['acc']['train'],
                          histories[experiment_id]['BEST']['loss']['test'],
                          histories[experiment_id]['BEST']['loss']['train'],
                          histories[experiment_id]['BEST']['dict_size']))
    
    
    if show_plots:
        for experiment_id in histories.keys():
            plt.plot(histories[experiment_id]['acc']['val'], label=experiment_id + ' val')
        plt.legend()
        plt.title('Validation Accuracy (Val only)')
        plt.show()

        for experiment_id in histories.keys():
            plt.plot(histories[experiment_id]['acc']['val'], label=experiment_id + ' val')
            plt.plot(histories[experiment_id]['acc']['train'], label=experiment_id + ' train')
        plt.legend()
        plt.title('Validation Accuracy (Val/Train)');
        plt.show()

        for experiment_id in histories.keys():
            plt.plot(histories[experiment_id]['loss']['val'], label=experiment_id  + ' val')
        plt.legend()
        plt.title('Validation Loss (Val only)');
        plt.show()

        for experiment_id in histories.keys():
            plt.plot(histories[experiment_id]['loss']['val'], label=experiment_id  + ' val')
            plt.plot(histories[experiment_id]['loss']['train'], label=experiment_id  + ' train')
        plt.legend()
        plt.title('Validation Loss (Val/Train)');
        plt.show()

def run_most_sumilars(func_most_similars, words_list, verbose = True, **kwargs):
    most_similars = {word: func_most_similars(word, **kwargs) for word in words_list}
    if verbose:
        for word, similars in most_similars.items():
            print('{}:'.format(word))
            print('\n'.join(map(str,similars)))
            print(' ')
    return most_similars
        

#https://stackoverflow.com/questions/4529815/saving-an-object-data-persistence/4529901
import pickle
def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

def load_object(filename):
    with open(filename, 'rb') as inp:
        return pickle.load(inp)

# sample usage
#company1 = [1,2,3,4,5]
#save_object(company1, '/kaggle/working/company1.pkl')
#del company
#company1 = load_object(filename)

In [6]:
WORDS_TO_SIMILAR = ['вклад', 'парадигма', 'лидерство', 'доверие', 'служение', 'семья', 'муж', 'мужчина', 'любовь', 'ненависть']
WORDS_ANALOGY = [
                {'positive': ['человек', 'лидер'], 'negative': ['совесть']}
                ]
WORDS_TO_PLOT = ['идея', 'один', 'между', 'который', 'отношения', 'могут', 'можно', 
                 'однако', 'человек', 'день', 'время', 'делать', 'должен', 'больше', 
                 'меньше', 'личность', 'характер', 'каждый', 'ненависть', 'лидерство',
                 'принцип', 'совесть', 'парадигма', 'доверие', 'верность', 'мужество', 
                 'ответственность', 'скромность', 'терпение', 'простота', 'трудолюбие', 
                 'справедливость', 'служение', 'вклад', 'качество', 'любовь', 'зависимость', 
                 'взаимозависимость', 'забота', 'поддержка', 'простота', 'цельность',
                 'честность', 'добросовестность', 'семья', 'муж', 'жена', 'супруг', 'мужчина']
#Здесь речь шла о таких свойствах, как цельность личности, скромность, верность, умеренность, 
#мужество, справедливость, терпеливость, трудолюбие, простота,

histories = {}

## Baseline

### Загрузка данных и подготовка корпуса

In [38]:
import random
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch import nn
from torch.nn import functional as F

from nltk.tokenize import sent_tokenize, word_tokenize

In [39]:
BOOK_DIRNAME = '/kaggle/input/my-private-datasets/CVBooks/rus/'
BOOK_LANGUAGE='russian'
corpus_paragraphs = []
for _, _, filenames in os.walk(BOOK_DIRNAME):
    for filename in filenames:
        #print(os.path.join(BOOK_DIRNAME, filename))
        with open(os.path.join(BOOK_DIRNAME, filename)) as file:
            corpus_paragraphs.extend([line.strip() for line in file])
            
corpus_paragraphs = list(filter(None, corpus_paragraphs)) #remove empty strings
print(len(corpus_paragraphs))
corpus_paragraphs[200:205]

In [40]:
#convert corpus of paragraphs to corpus of sentences
corpus_sentences = []
for paragraph in corpus_paragraphs:
    corpus_sentences.extend(sent_tokenize(paragraph, language=BOOK_LANGUAGE))

print(len(corpus_sentences))
corpus_sentences[200:205]

In [41]:
random.shuffle(corpus_sentences)
corpus_sentences[:5]

TRAIN_VAL_SPLIT = int(len(corpus_sentences) * 0.8)
train_source = corpus_sentences[:TRAIN_VAL_SPLIT]
test_source = corpus_sentences[TRAIN_VAL_SPLIT:]
print("Обучающая выборка", len(train_source))
print("Тестовая выборка", len(test_source))
print()
print('\n'.join(train_source[:5]))

In [42]:
# токенизируем
train_tokenized = tokenize_corpus(train_source, min_token_size = 3)
test_tokenized = tokenize_corpus(test_source, min_token_size = 3)
print('\n'.join(' '.join(sent) for sent in train_tokenized[:10]))

In [43]:
# строим словарь
vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=0.9, min_count=2, pad_word='<PAD>')
print("Размер словаря", len(vocabulary))
print(list(vocabulary.items())[:10])

In [44]:
# отображаем в номера токенов
train_token_ids = texts_to_token_ids(train_tokenized, vocabulary)
test_token_ids = texts_to_token_ids(test_tokenized, vocabulary)

print('\n'.join(' '.join(str(t) for t in sent)
                for sent in train_token_ids[:10]))

In [45]:
plt.hist([len(s) for s in train_token_ids], bins=20);
plt.title('Гистограмма длин предложений');

In [46]:
plt.hist([len(s) for s in train_token_ids], bins=20);
plt.yscale('log')
plt.title('Гистограмма длин предложений');

In [47]:
MAX_SENTENCE_LEN = 35
sum(np.array([len(s) for s in train_token_ids]) <= MAX_SENTENCE_LEN), sum(np.array([len(s) for s in train_token_ids]) > MAX_SENTENCE_LEN)

In [48]:
train_dataset = PaddedSequenceDataset(train_token_ids,
                                      np.zeros(len(train_token_ids)),
                                      out_len=MAX_SENTENCE_LEN)
test_dataset = PaddedSequenceDataset(test_token_ids,
                                     np.zeros(len(test_token_ids)),
                                     out_len=MAX_SENTENCE_LEN)
print(train_dataset[0])

## Алгоритм обучения - Skip Gram Negative Sampling

**Skip Gram** - предсказываем соседние слова по центральному слову

**Negative Sampling** - аппроксимация softmax

$$ W, D \in \mathbb{R}^{Vocab \times EmbSize} $$

$$ \sum_{CenterW_i} P(CtxW_{-2}, CtxW_{-1}, CtxW_{+1}, CtxW_{+2} | CenterW_i; W, D) \rightarrow \max_{W,D} $$

$$ P(CtxW_{-2}, CtxW_{-1}, CtxW_{+1}, CtxW_{+2} | CenterW_i; W, D) = \prod_j P(CtxW_j | CenterW_i; W, D) $$
    
$$ P(CtxW_j | CenterW_i; W, D) = \frac{e^{w_i \cdot d_j}} { \sum_{j=1}^{|V|} e^{w_i \cdot d_j}} = softmax \simeq \frac{e^{w_i \cdot d_j^+}} { \sum_{j=1}^{k} e^{w_i \cdot d_j^-}}, \quad k \ll |V| $$

In [49]:
def make_diag_mask(size, radius):
    """Квадратная матрица размера Size x Size с двумя полосами ширины radius вдоль главной диагонали"""
    idxs = torch.arange(size)
    abs_idx_diff = (idxs.unsqueeze(0) - idxs.unsqueeze(1)).abs()
    mask = ((abs_idx_diff <= radius) & (abs_idx_diff > 0)).float()
    return mask

make_diag_mask(10, 5)

**Negative Sampling** работает следующим образом - мы **максимизируем сумму вероятностей двух событий**: 

* "этот пример центрального слова вместе с контекстными словами взят **из тренировочной выборки**": $$ P(y=1 | CenterW_i; CtxW_j) = sigmoid(w_i \cdot d_j) = \frac{1}{1+e^{-w_i \cdot d_j}} $$

$$ \\ $$

* "этот пример центрального слова вместе со случайми контекстными словами **выдуман** ": $$ P(y=0 | CenterW_i; CtxW_{noise}) = 1 - P(y=1 | CenterW_i;  CtxW_{noise}) = \frac{1}{1+e^{w_i \cdot d_{noise}}} $$

$$ \\ $$

$$ NEG(CtxW_j, CenterW_i) = log(\frac{1}{1+e^{-w_i \cdot d_j}}) + \sum_{l=1}^{k}log(\frac{1}{1+e^{w_i \cdot d_{noise_l}}})  \rightarrow \max_{W,D} $$

In [ ]:
class SkipGramNegativeSamplingTrainer(nn.Module):
    def __init__(self, vocab_size, emb_size, sentence_len, radius=5, negative_samples_n=5):
        super().__init__()
        self.vocab_size = vocab_size
        self.negative_samples_n = negative_samples_n

        self.center_emb = nn.Embedding(self.vocab_size, emb_size, padding_idx=0)
        self.center_emb.weight.data.uniform_(-1.0 / emb_size, 1.0 / emb_size)
        self.center_emb.weight.data[0] = 0

        self.context_emb = nn.Embedding(self.vocab_size, emb_size, padding_idx=0)
        self.context_emb.weight.data.uniform_(-1.0 / emb_size, 1.0 / emb_size)
        self.context_emb.weight.data[0] = 0

        self.positive_sim_mask = make_diag_mask(sentence_len, radius)
    
    def forward(self, sentences):
        """sentences - Batch x MaxSentLength - идентификаторы токенов"""
        batch_size = sentences.shape[0]
        
        #получает на вход LongTensor с idx (т.е. индексами токенов), возвращает тензор + 1 измерения
        #в котором индексы заменены на соответсвующие им embedding'и (это центральные слова)
        #Итого(для batch=1): мы получаем тензор предложения фиксированной длины, где каждое слово 
        #заменено на embedding из центральных слов, все отсутсвующие слова (нет в словаре или 
        #закончилось реальное предложение), заменяются на embedding из 0
        center_embeddings = self.center_emb(sentences)  # Batch x MaxSentLength x EmbSize

        
        ### оценить сходство с настоящими соседними словами
        
        #получает на вход LongTensor с idx (т.е. индексами токенов), возвращает тензор + 1 измерения
        #в котором индексы заменены на соответсвующие им embedding'и, (это контекстные слова)
        #дополнительно транспонируем для целей последующего тензорного (матричного) умножения 
        #Итого(для batch=1): мы получаем тензор предложения фиксированной длины, где каждое слово 
        #заменено на embedding из контекстных слов, все отсутсвующие слова (нет в словаре или 
        #закончилось реальное предложение), заменяются на embedding из 0
        positive_context_embs = self.context_emb(sentences).permute(0, 2, 1)  # Batch x EmbSize x MaxSentLength
        
        #перемножение тензоров, по сути скалярное произведение эмбеддингов, 
        #Важно отметить, что изначально я проедполагал, что эта операция равносильна нахождению косинусных расстояний, 
        #т.к. на основе анализа итоговых эмбеддингов, сделал неверный вывод, что длина каждого из векторов уже здесь = 1 
        #(т.е. они сразу нормализуются в пределах каждого embedding (например внутри класса torch.nn.Embedding), 
        #но это не так, нормализация происходит уже после полного обучения модели, через передачу весов в конструктор 
        #созданного вручную класса Embedding)
        #Итого(для batch=1): мы получаем матрицу MaxSentLength x MaxSentLength, скалярных произведений, 
        #между векторами каждого центрального слова и каждого контекстного слова (значения [-inf; inf])
        positive_sims = torch.bmm(center_embeddings, positive_context_embs)  # Batch x MaxSentLength x MaxSentLength
        
        #преобразуем в "условные вероятности" через взятие сигмоиды, т.е. получаем как бы 
        #"условные вероятности" встретить пары слов вместе, по факту для каждой пары, скалярное произведение, 
        #обернутое в сигмоиду и как следствие в диапазон значений (0; 1)
        positive_probs = torch.sigmoid(positive_sims)

        
        ### увеличить оценку вероятности встретить эти пары слов вместе
        
        #переводим тензор self.positive_sim_mask на тот же девайс, на котором positive_sims
        positive_mask = self.positive_sim_mask.to(positive_sims.device)
        
        #.expand_as - Expand this tensor to the same size as other. 
        #self.expand_as(other) is equivalent to self.expand(other.size())
        #positive_probs * positive_mask - мы оставляем только позиции пересечения центральных слов в контекстными,
        #все остальные позиции зануляются
        #подсчитываем бинарную кросс энтропию вычисленных "условных вероятностей" (сигмоид) и целевых = 1 для всех
        #пересечений центральных и контекстных слов, все остальные позиции в обоих матрицах = 0 
        #Примечание: т.к. по умолчанию BCEloss в реализации torch высчитывает итоговое значение как 'mean', 
        #а не 'sum' из всех полученных, то количество 0 так же влияет на итоговое значение, имеет ли это какой 
        #то эффект, и измениться ли что то, если выставить reduction='sum', не очевидно и нужно проверять на практике
        #Примечание: для всех позиций, которые занулены, их эмбеддинги соответсвуют эмбеддинг-вектору с idx=0, для
        #для которого мы при создании мы указали паддинг nn.Embedding(..., padding_idx=0), это означает, что эти веса
        #фиксированы, и не подлежат изменению через градиентных шаг
        #
        #Итого: важно понимать, что если бы оптимизировали только данную loss функцию, без отрицательных примеров, 
        #которые идут ниже, то, все сводилось бы к тому, что минимальное значение loss было бы, если бы мы все 
        #вектора (и центральных слов и контекстных) устремили бы в бесконечность, в одном направлении (например всем
        #их весам присвоили бы значение inf или любые подобные варианты)
        positive_loss = F.binary_cross_entropy(positive_probs * positive_mask,
                                               positive_mask.expand_as(positive_probs))

        
        ### выбрать случайные "отрицательные" слова
        negative_words = torch.randint(1, self.vocab_size,
                                       size=(batch_size, self.negative_samples_n),
                                       device=sentences.device)  # Batch x NegSamplesN
        negative_context_embs = self.context_emb(negative_words).permute(0, 2, 1)  # Batch x EmbSize x NegSamplesN
        negative_sims = torch.bmm(center_embeddings, negative_context_embs)  # Batch x MaxSentLength x NegSamplesN
        
        ### уменьшить оценку вероятность встретить эти пары слов вместе
        #Важно отметить, что BCEWithLogitsLoss расвносильна последовательному применению Sigmoid -> BCELoss
        #но в реализации torch она является более численно стабильной, чем раздельное применение
        #Итого: здесь все целевые (target) значения = 0, и если бы мы минимизировали только эту loss функцию, то минимальное
        #ее значение было бы, если бы мы устремили все вектора центральных слов в бесконечность одного направления, 
        #а вектора контекстных слов в бесконечность противоположного направления
        negative_loss = F.binary_cross_entropy_with_logits(negative_sims,
                                                           negative_sims.new_zeros(negative_sims.shape))

        return positive_loss + negative_loss


def no_loss(pred, target):
    """Фиктивная функция потерь - когда модель сама считает функцию потерь"""
    return pred

## Обучение

In [ ]:
# trainer = SkipGramNegativeSamplingTrainer(len(vocabulary), 100, MAX_SENTENCE_LEN,
#                                           radius=5, negative_samples_n=25)

In [ ]:
# train_history, best_model = train_eval_loop(trainer,
#                                             train_dataset,
#                                             test_dataset,
#                                             no_loss,
#                                             lr=1e-2,
#                                             epoch_n=10,
#                                             batch_size=64,
#                                             device='cpu',
#                                             early_stopping_patience=4,
#                                             max_batches_per_epoch_train=10000,
#                                             max_batches_per_epoch_val=len(test_dataset),
#                                             lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=1, verbose=True),
#                                             no_calculate_accuracy = True)

In [ ]:
# # Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
# torch.save(trainer.state_dict(), './sgns.pth')

In [ ]:
# # Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
# trainer.load_state_dict(torch.load('./sgns.pth'))

## Исследуем характеристики полученных векторов

In [ ]:
# embeddings = Embeddings(trainer.center_emb.weight.detach().cpu().numpy(), vocabulary)

In [ ]:
# run_most_sumilars(embeddings.most_similar,WORDS_TO_SIMILAR);

In [ ]:
# print(WORDS_ANALOGY[0])
# embeddings.most_similar(**WORDS_ANALOGY[0], topk=10, with_mean=True)

In [ ]:
# #решение семантической пропорции — то есть задача аналогии
# #word2 - word1 + word3
# embeddings.analogy('совесть', 'лидер', 'человек')

In [ ]:
# words_to_plot = [w for w in WORDS_TO_PLOT if w in embeddings.word2id]
# test_vectors = embeddings.get_vectors(*words_to_plot)
# print(test_vectors.shape)
# fig, ax = plt.subplots()
# fig.set_size_inches((10, 10))
# plot_vectors(test_vectors, WORDS_TO_PLOT, how='svd', ax=ax)

## Обучение Word2Vec с помощью Gensim

In [ ]:
exp_name = 'base_gensim'

hyps = {
    'min_token_size': 3,
    'min_count': 2,
    'emb_size': 100,
    'rwindow': 5,
}

histories[exp_name] = {
    'hyps': hyps
}

In [ ]:
import gensim

In [ ]:
word2vec = gensim.models.Word2Vec(sentences=train_tokenized, vector_size=100,
                                  window=5, min_count=2, workers=4,
                                  sg=1, epochs=10)

In [ ]:
histories[exp_name]['most_similars'] = run_most_sumilars(word2vec.wv.most_similar,WORDS_TO_SIMILAR)

In [ ]:
print(WORDS_ANALOGY[0])
word2vec.wv.most_similar(**WORDS_ANALOGY[0])

In [ ]:
gensim_words = [w for w in WORDS_TO_PLOT if w in word2vec.wv.key_to_index]
gensim_vectors = np.stack([word2vec.wv[w] for w in gensim_words])
histories[exp_name]['test_vectors'] = {word:vec for word,vec in zip(gensim_words, gensim_vectors)}
fig, ax = plt.subplots()
fig.set_size_inches((10, 10))
plot_vectors(gensim_vectors, gensim_words, how='svd', ax=ax)

## Загрузка предобученного Word2Vec

Источники готовых векторов:

https://rusvectores.org/ru/ - для русского языка

https://wikipedia2vec.github.io/wikipedia2vec/pretrained/ - много разных языков

In [ ]:
# import gensim.downloader as api

In [ ]:
# available_models = api.info()['models'].keys()
# print('\n'.join(available_models))

In [ ]:
# pretrained = api.load('word2vec-ruscorpora-300')  # > 1.5 GB!

In [ ]:
# pretrained.most_similar('служение_NOUN')

In [ ]:
# pretrained.most_similar(positive=['man', 'queen'], negative=['king'])

In [ ]:
# pretrained_words = [w for w in test_words if w in pretrained.key_to_index]
# pretrained_vectors = np.stack([pretrained[w] for w in pretrained_words])

In [ ]:
# [pretrained[w] for w in pretrained_words]

In [ ]:
# fig, ax = plt.subplots()
# fig.set_size_inches((10, 10))
# plot_vectors(pretrained_vectors, test_words, how='svd', ax=ax)

## Эксперименты с различными параметрами

In [ ]:
#hyperparameters
exp_name = 'newbaseline'
hyps = {
    'min_token_size': 3,
    'min_count': 2,
    'max_doc_freq': 0.9, 

    'MAX_SENTENCE_LEN': 35,
    'emb_size': 100,
    'rwindow': 5,
    
    'neg_sampl_n': 25,
    'lr': 1e-2,
    'epoch_n': 50,
    'batch': 2048,
    'device': 'cpu',
    'stop_pat': 4,
    'lr_sched_pat': 1
}
    
histories[exp_name] = {
    'hyps': hyps
}

# токенизируем
train_tokenized = tokenize_corpus(train_source, min_token_size=hyps['min_token_size'])
test_tokenized = tokenize_corpus(test_source, min_token_size=hyps['min_token_size'])
print('\n'.join(' '.join(sent) for sent in train_tokenized[:10]))

# строим словарь
vocabulary, word_doc_freq = build_vocabulary(train_tokenized, 
                                             max_doc_freq=hyps['max_doc_freq'], 
                                             min_count=hyps['min_count'], pad_word='<PAD>')
histories[exp_name]['vocab_size'] = len(vocabulary)

print("Размер словаря", len(vocabulary))
print(list(vocabulary.items())[:10])

# отображаем в номера токенов
train_token_ids = texts_to_token_ids(train_tokenized, vocabulary)
test_token_ids = texts_to_token_ids(test_tokenized, vocabulary)

# plt.hist([len(s) for s in train_token_ids], bins=20);
# plt.title('Гистограмма длин предложений');

print('MAX_SENTENCE_LEN = {}, Sentences counts: (<=) = {}, (>) = {}'.format(hyps['MAX_SENTENCE_LEN'],
                                                sum(np.array([len(s) for s in train_token_ids]) <= hyps['MAX_SENTENCE_LEN']),
                                                sum(np.array([len(s) for s in train_token_ids]) > hyps['MAX_SENTENCE_LEN'])))

train_dataset = PaddedSequenceDataset(train_token_ids,
                                      np.zeros(len(train_token_ids)),
                                      out_len=hyps['MAX_SENTENCE_LEN'])
test_dataset = PaddedSequenceDataset(test_token_ids,
                                     np.zeros(len(test_token_ids)),
                                     out_len=hyps['MAX_SENTENCE_LEN'])
#print(train_dataset[0])

### Обучение
trainer = SkipGramNegativeSamplingTrainer(len(vocabulary), hyps['emb_size'], hyps['MAX_SENTENCE_LEN'],
                                          radius=hyps['rwindow'], negative_samples_n=hyps['neg_sampl_n'])

histories[exp_name]['train_history'], best_model = train_eval_loop(
                                            trainer,
                                            train_dataset,
                                            test_dataset,
                                            no_loss,
                                            lr=hyps['lr'],
                                            epoch_n=hyps['epoch_n'],
                                            batch_size=hyps['batch'],
                                            device=hyps['device'],
                                            early_stopping_patience=hyps['stop_pat'],
                                            max_batches_per_epoch_train=10000,
                                            max_batches_per_epoch_val=len(test_dataset),
                                            lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, 
                                                                                                                       patience=hyps['lr_sched_pat'], 
                                                                                                                       verbose=True),
                                            no_calculate_accuracy = True)

### Исследуем характеристики полученных векторов
embeddings = Embeddings(trainer.center_emb.weight.detach().cpu().numpy(), vocabulary)

print('MOST_SIMILAR:')
histories[exp_name]['most_similars'] = run_most_sumilars(embeddings.most_similar,WORDS_TO_SIMILAR)

print('WORDS_ANALOGY:')
print(WORDS_ANALOGY[0])
print('\n'.join(map(str,embeddings.most_similar(**WORDS_ANALOGY[0], topk=10, with_mean=True))))

print('\nWORDS_ON_PLOT:')
words_to_plot = [w for w in WORDS_TO_PLOT if w in embeddings.word2id]
test_vectors = embeddings.get_vectors(*words_to_plot)
print(test_vectors.shape)

histories[exp_name]['test_vectors'] = {word:vec for word,vec in zip(WORDS_TO_PLOT, test_vectors)}

fig, ax = plt.subplots()
fig.set_size_inches((10, 10))
plot_vectors(np.array(list(histories[exp_name]['test_vectors'].values())), 
             histories[exp_name]['test_vectors'].keys(), 
             how='svd', ax=ax)

In [ ]:
# #hyperparameters
# exp_name = 'batch=2048'
# hyps = {
#     'min_token_size': 4,
#     'min_count': 2,
#     'max_doc_freq': 0.9, 

#     'MAX_SENTENCE_LEN': 35,
#     'emb_size': 100,
#     'rwindow': 5,
    
#     'neg_sampl_n': 25,
#     'lr': 1e-2,
#     'epoch_n': 10,
#     'batch': 2048,
#     'device': 'cpu',
#     'stop_pat': 4,
#     'lr_sched_pat': 1
# }
    
# histories[exp_name] = {
#     'hyps': hyps
# }

# # токенизируем
# train_tokenized = tokenize_corpus(train_source, min_token_size=hyps['min_token_size'])
# test_tokenized = tokenize_corpus(test_source, min_token_size=hyps['min_token_size'])
# print('\n'.join(' '.join(sent) for sent in train_tokenized[:10]))

# # строим словарь
# vocabulary, word_doc_freq = build_vocabulary(train_tokenized, 
#                                              max_doc_freq=hyps['max_doc_freq'], 
#                                              min_count=hyps['min_count'], pad_word='<PAD>')
# histories[exp_name]['vocab_size'] = len(vocabulary)

# print("Размер словаря", len(vocabulary))
# print(list(vocabulary.items())[:10])

# # отображаем в номера токенов
# train_token_ids = texts_to_token_ids(train_tokenized, vocabulary)
# test_token_ids = texts_to_token_ids(test_tokenized, vocabulary)

# # plt.hist([len(s) for s in train_token_ids], bins=20);
# # plt.title('Гистограмма длин предложений');

# print('MAX_SENTENCE_LEN = {}, Sentences counts: (<=) = {}, (>) = {}'.format(hyps['MAX_SENTENCE_LEN'],
#                                                 sum(np.array([len(s) for s in train_token_ids]) <= hyps['MAX_SENTENCE_LEN']),
#                                                 sum(np.array([len(s) for s in train_token_ids]) > hyps['MAX_SENTENCE_LEN'])))

# train_dataset = PaddedSequenceDataset(train_token_ids,
#                                       np.zeros(len(train_token_ids)),
#                                       out_len=hyps['MAX_SENTENCE_LEN'])
# test_dataset = PaddedSequenceDataset(test_token_ids,
#                                      np.zeros(len(test_token_ids)),
#                                      out_len=hyps['MAX_SENTENCE_LEN'])
# #print(train_dataset[0])

# ### Обучение
# trainer = SkipGramNegativeSamplingTrainer(len(vocabulary), hyps['emb_size'], hyps['MAX_SENTENCE_LEN'],
#                                           radius=hyps['rwindow'], negative_samples_n=hyps['neg_sampl_n'])

# histories[exp_name]['train_history'], best_model = train_eval_loop(
#                                             trainer,
#                                             train_dataset,
#                                             test_dataset,
#                                             no_loss,
#                                             lr=hyps['lr'],
#                                             epoch_n=hyps['epoch_n'],
#                                             batch_size=hyps['batch'],
#                                             device=hyps['device'],
#                                             early_stopping_patience=hyps['stop_pat'],
#                                             max_batches_per_epoch_train=10000,
#                                             max_batches_per_epoch_val=len(test_dataset),
#                                             lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, 
#                                                                                                                        patience=hyps['lr_sched_pat'], 
#                                                                                                                        verbose=True),
#                                             no_calculate_accuracy = True)



# ### Исследуем характеристики полученных векторов
# embeddings = Embeddings(trainer.center_emb.weight.detach().cpu().numpy(), vocabulary)

# print('MOST_SIMILAR:')
# histories[exp_name]['most_similars'] = run_most_sumilars(embeddings.most_similar,WORDS_TO_SIMILAR)

# print('WORDS_ANALOGY:')
# print(WORDS_ANALOGY[0])
# print('\n'.join(map(str,embeddings.most_similar(**WORDS_ANALOGY[0], topk=10, with_mean=True))))

# print('\nWORDS_ON_PLOT:')
# words_to_plot = [w for w in WORDS_TO_PLOT if w in embeddings.word2id]
# test_vectors = embeddings.get_vectors(*words_to_plot)
# print(test_vectors.shape)

# histories[exp_name]['test_vectors'] = {word:vec for word,vec in zip(WORDS_TO_PLOT, test_vectors)}

# fig, ax = plt.subplots()
# fig.set_size_inches((10, 10))
# plot_vectors(np.array(list(histories[exp_name]['test_vectors'].values())), 
#              histories[exp_name]['test_vectors'].keys(), 
#              how='svd', ax=ax)

In [ ]:
# #hyperparameters
# exp_name = 'batch=2048_rwindow=10'
# hyps = {
#     'min_token_size': 4,
#     'min_count': 2,
#     'max_doc_freq': 0.9, 

#     'MAX_SENTENCE_LEN': 35,
#     'emb_size': 100,
#     'rwindow': 10,
    
#     'neg_sampl_n': 25,
#     'lr': 1e-2,
#     'epoch_n': 10,
#     'batch': 2048,
#     'device': 'cpu',
#     'stop_pat': 4,
#     'lr_sched_pat': 1
# }
    
# histories[exp_name] = {
#     'hyps': hyps
# }

# # токенизируем
# train_tokenized = tokenize_corpus(train_source, min_token_size=hyps['min_token_size'])
# test_tokenized = tokenize_corpus(test_source, min_token_size=hyps['min_token_size'])
# print('\n'.join(' '.join(sent) for sent in train_tokenized[:10]))

# # строим словарь
# vocabulary, word_doc_freq = build_vocabulary(train_tokenized, 
#                                              max_doc_freq=hyps['max_doc_freq'], 
#                                              min_count=hyps['min_count'], pad_word='<PAD>')
# histories[exp_name]['vocab_size'] = len(vocabulary)

# print("Размер словаря", len(vocabulary))
# print(list(vocabulary.items())[:10])

# # отображаем в номера токенов
# train_token_ids = texts_to_token_ids(train_tokenized, vocabulary)
# test_token_ids = texts_to_token_ids(test_tokenized, vocabulary)

# # plt.hist([len(s) for s in train_token_ids], bins=20);
# # plt.title('Гистограмма длин предложений');

# print('MAX_SENTENCE_LEN = {}, Sentences counts: (<=) = {}, (>) = {}'.format(hyps['MAX_SENTENCE_LEN'],
#                                                 sum(np.array([len(s) for s in train_token_ids]) <= hyps['MAX_SENTENCE_LEN']),
#                                                 sum(np.array([len(s) for s in train_token_ids]) > hyps['MAX_SENTENCE_LEN'])))

# train_dataset = PaddedSequenceDataset(train_token_ids,
#                                       np.zeros(len(train_token_ids)),
#                                       out_len=hyps['MAX_SENTENCE_LEN'])
# test_dataset = PaddedSequenceDataset(test_token_ids,
#                                      np.zeros(len(test_token_ids)),
#                                      out_len=hyps['MAX_SENTENCE_LEN'])
# #print(train_dataset[0])

# ### Обучение
# trainer = SkipGramNegativeSamplingTrainer(len(vocabulary), hyps['emb_size'], hyps['MAX_SENTENCE_LEN'],
#                                           radius=hyps['rwindow'], negative_samples_n=hyps['neg_sampl_n'])

# histories[exp_name]['train_history'], best_model = train_eval_loop(
#                                             trainer,
#                                             train_dataset,
#                                             test_dataset,
#                                             no_loss,
#                                             lr=hyps['lr'],
#                                             epoch_n=hyps['epoch_n'],
#                                             batch_size=hyps['batch'],
#                                             device=hyps['device'],
#                                             early_stopping_patience=hyps['stop_pat'],
#                                             max_batches_per_epoch_train=10000,
#                                             max_batches_per_epoch_val=len(test_dataset),
#                                             lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, 
#                                                                                                                        patience=hyps['lr_sched_pat'], 
#                                                                                                                        verbose=True),
#                                             no_calculate_accuracy = True)



# ### Исследуем характеристики полученных векторов
# embeddings = Embeddings(trainer.center_emb.weight.detach().cpu().numpy(), vocabulary)

# print('MOST_SIMILAR:')
# histories[exp_name]['most_similars'] = run_most_sumilars(embeddings.most_similar,WORDS_TO_SIMILAR)

# print('WORDS_ANALOGY:')
# print(WORDS_ANALOGY[0])
# print('\n'.join(map(str,embeddings.most_similar(**WORDS_ANALOGY[0], topk=10, with_mean=True))))

# print('\nWORDS_ON_PLOT:')
# words_to_plot = [w for w in WORDS_TO_PLOT if w in embeddings.word2id]
# test_vectors = embeddings.get_vectors(*words_to_plot)
# print(test_vectors.shape)

# histories[exp_name]['test_vectors'] = {word:vec for word,vec in zip(WORDS_TO_PLOT, test_vectors)}

# fig, ax = plt.subplots()
# fig.set_size_inches((10, 10))
# plot_vectors(np.array(list(histories[exp_name]['test_vectors'].values())), 
#              histories[exp_name]['test_vectors'].keys(), 
#              how='svd', ax=ax)

### Эксперимент - убрать разбиение текстов на предложения и увеличить окно

In [ ]:
# def make_diag_mask(size, radius, with_padding = False):
#     """Квадратная матрица размера Size x (Size - 2*radius)с двумя полосами ширины radius вдоль главной диагонали
#        upd(wis) добавил возможность указать padding для целей задачи, когда мы не бьем текст по предложениям"""
#     padding = radius if with_padding else 0
#     idxs_col = torch.arange(size)
#     idxs_row = torch.arange(size - padding * 2) + padding
#     abs_idx_diff = (idxs_col.unsqueeze(0) - idxs_row.unsqueeze(1)).abs()
#     mask = ((abs_idx_diff <= radius) & (abs_idx_diff > 0)).float()
#     return mask

# make_diag_mask(18, 3, True)


# class SkipGramNegativeSamplingTrainer(nn.Module):
#     #Modified for work with paddings
#     def __init__(self, vocab_size, emb_size, sentence_len, radius=5, negative_samples_n=5, with_padding=False):
#         super().__init__()
#         self.vocab_size = vocab_size
#         self.negative_samples_n = negative_samples_n

#         self.center_emb = nn.Embedding(self.vocab_size, emb_size, padding_idx=0)
#         self.center_emb.weight.data.uniform_(-1.0 / emb_size, 1.0 / emb_size)
#         self.center_emb.weight.data[0] = 0

#         self.context_emb = nn.Embedding(self.vocab_size, emb_size, padding_idx=0)        
#         self.context_emb.weight.data.uniform_(-1.0 / emb_size, 1.0 / emb_size)
#         self.context_emb.weight.data[0] = 0
        
#         self.positive_sim_mask = make_diag_mask(sentence_len, radius, with_padding=with_padding)
        
#         self.with_padding = with_padding
#         self.padding = radius if with_padding else 0
    
#     def forward(self, sentences):
#         """sentences - Batch x MaxSentLength - идентификаторы токенов"""
#         batch_size = sentences.shape[0]
        
#         #если мы используем padding, то нам не нужно брать в качестве центральных слов те, которые отстоят
#         #от краев (начала/конца) на размер этого паддинга, т.е. вырезаем центр тензора без паддинга
#         if self.with_padding:
#             #получает на вход LongTensor с idx (т.е. индексами токенов), возвращает тензор + 1 измерения
#             #в котором индексы заменены на соответсвующие им embedding'и
#             center_embeddings = self.center_emb(sentences[:,self.padding:-self.padding])  # Batch x (MaxSentLength-2*padding) x EmbSize
#         else:
#             center_embeddings = self.center_emb(sentences)  # Batch x MaxSentLength x EmbSize
        

#         # оценить сходство с настоящими соседними словами
#         positive_context_embs = self.context_emb(sentences).permute(0, 2, 1)  # Batch x EmbSize x MaxSentLength
#         positive_sims = torch.bmm(center_embeddings, positive_context_embs)  # Batch x (MaxSentLength-2*padding) x (MaxSentLength-2*padding)
#         positive_probs = torch.sigmoid(positive_sims)

#         # увеличить оценку вероятности встретить эти пары слов вместе
#         positive_mask = self.positive_sim_mask.to(positive_sims.device)
#         positive_loss = F.binary_cross_entropy(positive_probs * positive_mask,
#                                                positive_mask.expand_as(positive_probs))

#         # выбрать случайные "отрицательные" слова
#         negative_words = torch.randint(1, self.vocab_size,
#                                        size=(batch_size, self.negative_samples_n),
#                                        device=sentences.device)  # Batch x NegSamplesN
#         negative_context_embs = self.context_emb(negative_words).permute(0, 2, 1)  # Batch x EmbSize x NegSamplesN
#         negative_sims = torch.bmm(center_embeddings, negative_context_embs)  # Batch x (MaxSentLength-2*padding) x NegSamplesN
        
#         # уменьшить оценку вероятность встретить эти пары слов вместе
#         negative_loss = F.binary_cross_entropy_with_logits(negative_sims,
#                                                            negative_sims.new_zeros(negative_sims.shape))

#         return positive_loss + negative_loss


# def no_loss(pred, target):
#     """Фиктивная функция потерь - когда модель сама считает функцию потерь"""
#     return pred

In [ ]:
# #hyperparameters
# exp_name = 'no_sentenses_batch=2048'
# hyps = {
#     'min_token_size': 4,
#     'min_count': 2,
#     'max_doc_freq': 0.9, 

#     'MAX_SENTENCE_LEN': 35,
#     'emb_size': 100,
#     'rwindow': 5,
#     'padding': True,
    
#     'neg_sampl_n': 25,
#     'lr': 1e-2,
#     'epoch_n': 10,
#     'batch': 2048,
#     'device': 'cpu',
#     'stop_pat': 4,
#     'lr_sched_pat': 1
# }
    
# histories[exp_name] = {
#     'hyps': hyps
# }

# BOOK_DIRNAME = '/kaggle/input/my-private-datasets/CVBooks/rus/'
# BOOK_LANGUAGE='russian'
# corpus_paragraphs = []
# for _, _, filenames in os.walk(BOOK_DIRNAME):
#     for filename in filenames:
#         #print(os.path.join(BOOK_DIRNAME, filename))
#         with open(os.path.join(BOOK_DIRNAME, filename)) as file:
#             corpus_paragraphs.extend([line.strip() for line in file])
            
# corpus_paragraphs = list(filter(None, corpus_paragraphs)) #remove empty strings
# print(len(corpus_paragraphs))
# corpus_paragraphs[200:205]

# #Для текущего эксперимента склеиваем все параграфы в единый текст.
# plain_corpus = ' '.join(corpus_paragraphs)
# #plain_corpus[1000:2000]

# #convert plain corpus to corpus of blocks with padding
# plain_corpus_tokenised = tokenize_corpus([plain_corpus], min_token_size=hyps['min_token_size'])[0]
# print(len(plain_corpus_tokenised))

# #partition into blocks
# block_len = hyps['MAX_SENTENCE_LEN'] - 2 * hyps['rwindow']

# corpus_blocks = []
# corpus_blocks.append(plain_corpus_tokenised[:hyps['rwindow']]) #only for padding

# for i in range(hyps['rwindow'], len(plain_corpus_tokenised) - block_len - hyps['rwindow']*2 + 1, block_len):
#     corpus_blocks.append(plain_corpus_tokenised[i:i+block_len])

# corpus_blocks.append(plain_corpus_tokenised[i:i+hyps['rwindow']]) #only for padding

# print(len(corpus_blocks) - 2)

# #add paddings
# padding = hyps['rwindow']
# for i in range(1, len(corpus_blocks) - 2):
#     corpus_blocks[i] = corpus_blocks[i-1][-padding:] + corpus_blocks[i] + corpus_blocks[i+1][:padding]
# corpus_blocks = corpus_blocks[1:-2]
# #print(corpus_blocks[10:14])

# #split to train/test
# random.shuffle(corpus_blocks)

# TRAIN_VAL_SPLIT = int(len(corpus_blocks) * 0.8)
# train_tokenized = corpus_blocks[:TRAIN_VAL_SPLIT]
# test_tokenized = corpus_blocks[TRAIN_VAL_SPLIT:]
# print("Обучающая выборка", len(train_tokenized))
# print("Тестовая выборка", len(test_tokenized))

# # строим словарь, используем для построения train_tokenized с удаленными паддингами
# vocabulary, word_doc_freq = build_vocabulary([block[padding:-padding] for block in train_tokenized], 
#                                              max_doc_freq=hyps['max_doc_freq'], 
#                                              min_count=hyps['min_count'], pad_word='<PAD>')
# histories[exp_name]['vocab_size'] = len(vocabulary)

# print("Размер словаря", len(vocabulary))
# print(list(vocabulary.items())[:10])

# # отображаем в номера токенов
# train_token_ids = texts_to_token_ids(train_tokenized, vocabulary)
# test_token_ids = texts_to_token_ids(test_tokenized, vocabulary)


# train_dataset = PaddedSequenceDataset(train_token_ids,
#                                       np.zeros(len(train_token_ids)),
#                                       out_len=hyps['MAX_SENTENCE_LEN'])
# test_dataset = PaddedSequenceDataset(test_token_ids,
#                                      np.zeros(len(test_token_ids)),
#                                      out_len=hyps['MAX_SENTENCE_LEN'])
# print(train_dataset[0], len(train_dataset[0][0]))

# ### Обучение
# trainer = SkipGramNegativeSamplingTrainer(len(vocabulary), hyps['emb_size'], hyps['MAX_SENTENCE_LEN'],
#                                           radius=hyps['rwindow'], negative_samples_n=hyps['neg_sampl_n'], with_padding=True)

# histories[exp_name]['train_history'], best_model = train_eval_loop(
#                                             trainer,
#                                             train_dataset,
#                                             test_dataset,
#                                             no_loss,
#                                             lr=hyps['lr'],
#                                             epoch_n=hyps['epoch_n'],
#                                             batch_size=hyps['batch'],
#                                             device=hyps['device'],
#                                             early_stopping_patience=hyps['stop_pat'],
#                                             max_batches_per_epoch_train=10000,
#                                             max_batches_per_epoch_val=len(test_dataset),
#                                             lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, 
#                                                                                                                        patience=hyps['lr_sched_pat'], 
#                                                                                                                        verbose=True),
#                                             no_calculate_accuracy = True)



# ### Исследуем характеристики полученных векторов
# embeddings = Embeddings(trainer.center_emb.weight.detach().cpu().numpy(), vocabulary)

# print('MOST_SIMILAR:')
# histories[exp_name]['most_similars'] = run_most_sumilars(embeddings.most_similar,WORDS_TO_SIMILAR)

# print('WORDS_ANALOGY:')
# print(WORDS_ANALOGY[0])
# print('\n'.join(map(str,embeddings.most_similar(**WORDS_ANALOGY[0], topk=10, with_mean=True))))

# print('\nWORDS_ON_PLOT:')
# words_to_plot = [w for w in WORDS_TO_PLOT if w in embeddings.word2id]
# test_vectors = embeddings.get_vectors(*words_to_plot)
# print(test_vectors.shape)

# histories[exp_name]['test_vectors'] = {word:vec for word,vec in zip(WORDS_TO_PLOT, test_vectors)}

# fig, ax = plt.subplots()
# fig.set_size_inches((10, 10))
# plot_vectors(np.array(list(histories[exp_name]['test_vectors'].values())), 
#              histories[exp_name]['test_vectors'].keys(), 
#              how='svd', ax=ax)

In [ ]:
# #hyperparameters
# exp_name = 'no_sentenses_rwindow=8_batch=2048'
# hyps = {
#     'min_token_size': 4,
#     'min_count': 2,
#     'max_doc_freq': 0.9, 

#     'MAX_SENTENCE_LEN': 35,
#     'emb_size': 100,
#     'rwindow': 8,
#     'padding': True,
    
#     'neg_sampl_n': 25,
#     'lr': 1e-2,
#     'epoch_n': 10,
#     'batch': 2048,
#     'device': 'cpu',
#     'stop_pat': 4,
#     'lr_sched_pat': 1
# }
    
# histories[exp_name] = {
#     'hyps': hyps
# }

# BOOK_DIRNAME = '/kaggle/input/my-private-datasets/CVBooks/rus/'
# BOOK_LANGUAGE='russian'
# corpus_paragraphs = []
# for _, _, filenames in os.walk(BOOK_DIRNAME):
#     for filename in filenames:
#         #print(os.path.join(BOOK_DIRNAME, filename))
#         with open(os.path.join(BOOK_DIRNAME, filename)) as file:
#             corpus_paragraphs.extend([line.strip() for line in file])
            
# corpus_paragraphs = list(filter(None, corpus_paragraphs)) #remove empty strings
# print(len(corpus_paragraphs))
# corpus_paragraphs[200:205]

# #Для текущего эксперимента склеиваем все параграфы в единый текст.
# plain_corpus = ' '.join(corpus_paragraphs)
# #plain_corpus[1000:2000]

# #convert plain corpus to corpus of blocks with padding
# plain_corpus_tokenised = tokenize_corpus([plain_corpus], min_token_size=hyps['min_token_size'])[0]
# print(len(plain_corpus_tokenised))

# #partition into blocks
# block_len = hyps['MAX_SENTENCE_LEN'] - 2 * hyps['rwindow']

# corpus_blocks = []
# corpus_blocks.append(plain_corpus_tokenised[:hyps['rwindow']]) #only for padding

# for i in range(hyps['rwindow'], len(plain_corpus_tokenised) - block_len - hyps['rwindow']*2 + 1, block_len):
#     corpus_blocks.append(plain_corpus_tokenised[i:i+block_len])

# corpus_blocks.append(plain_corpus_tokenised[i:i+hyps['rwindow']]) #only for padding

# print(len(corpus_blocks) - 2)

# #add paddings
# padding = hyps['rwindow']
# for i in range(1, len(corpus_blocks) - 2):
#     corpus_blocks[i] = corpus_blocks[i-1][-padding:] + corpus_blocks[i] + corpus_blocks[i+1][:padding]
# corpus_blocks = corpus_blocks[1:-2]
# #print(corpus_blocks[10:14])

# #split to train/test
# random.shuffle(corpus_blocks)

# TRAIN_VAL_SPLIT = int(len(corpus_blocks) * 0.8)
# train_tokenized = corpus_blocks[:TRAIN_VAL_SPLIT]
# test_tokenized = corpus_blocks[TRAIN_VAL_SPLIT:]
# print("Обучающая выборка", len(train_tokenized))
# print("Тестовая выборка", len(test_tokenized))

# # строим словарь, используем для построения train_tokenized с удаленными паддингами
# vocabulary, word_doc_freq = build_vocabulary([block[padding:-padding] for block in train_tokenized], 
#                                              max_doc_freq=hyps['max_doc_freq'], 
#                                              min_count=hyps['min_count'], pad_word='<PAD>')
# histories[exp_name]['vocab_size'] = len(vocabulary)

# print("Размер словаря", len(vocabulary))
# print(list(vocabulary.items())[:10])

# # отображаем в номера токенов
# train_token_ids = texts_to_token_ids(train_tokenized, vocabulary)
# test_token_ids = texts_to_token_ids(test_tokenized, vocabulary)


# train_dataset = PaddedSequenceDataset(train_token_ids,
#                                       np.zeros(len(train_token_ids)),
#                                       out_len=hyps['MAX_SENTENCE_LEN'])
# test_dataset = PaddedSequenceDataset(test_token_ids,
#                                      np.zeros(len(test_token_ids)),
#                                      out_len=hyps['MAX_SENTENCE_LEN'])
# print(train_dataset[0], len(train_dataset[0][0]))

# ### Обучение
# trainer = SkipGramNegativeSamplingTrainer(len(vocabulary), hyps['emb_size'], hyps['MAX_SENTENCE_LEN'],
#                                           radius=hyps['rwindow'], negative_samples_n=hyps['neg_sampl_n'], with_padding=True)

# histories[exp_name]['train_history'], best_model = train_eval_loop(
#                                             trainer,
#                                             train_dataset,
#                                             test_dataset,
#                                             no_loss,
#                                             lr=hyps['lr'],
#                                             epoch_n=hyps['epoch_n'],
#                                             batch_size=hyps['batch'],
#                                             device=hyps['device'],
#                                             early_stopping_patience=hyps['stop_pat'],
#                                             max_batches_per_epoch_train=10000,
#                                             max_batches_per_epoch_val=len(test_dataset),
#                                             lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, 
#                                                                                                                        patience=hyps['lr_sched_pat'], 
#                                                                                                                        verbose=True),
#                                             no_calculate_accuracy = True)



# ### Исследуем характеристики полученных векторов
# embeddings = Embeddings(trainer.center_emb.weight.detach().cpu().numpy(), vocabulary)

# print('MOST_SIMILAR:')
# histories[exp_name]['most_similars'] = run_most_sumilars(embeddings.most_similar,WORDS_TO_SIMILAR)

# print('WORDS_ANALOGY:')
# print(WORDS_ANALOGY[0])
# print('\n'.join(map(str,embeddings.most_similar(**WORDS_ANALOGY[0], topk=10, with_mean=True))))

# print('\nWORDS_ON_PLOT:')
# words_to_plot = [w for w in WORDS_TO_PLOT if w in embeddings.word2id]
# test_vectors = embeddings.get_vectors(*words_to_plot)
# print(test_vectors.shape)

# histories[exp_name]['test_vectors'] = {word:vec for word,vec in zip(WORDS_TO_PLOT, test_vectors)}

# fig, ax = plt.subplots()
# fig.set_size_inches((10, 10))
# plot_vectors(np.array(list(histories[exp_name]['test_vectors'].values())), 
#              histories[exp_name]['test_vectors'].keys(), 
#              how='svd', ax=ax)

In [ ]:
# #hyperparameters
# exp_name = 'no_sentenses_rwindow=12_batch=2048'
# hyps = {
#     'min_token_size': 4,
#     'min_count': 2,
#     'max_doc_freq': 0.9, 

#     'MAX_SENTENCE_LEN': 35,
#     'emb_size': 100,
#     'rwindow': 12,
#     'padding': True,
    
#     'neg_sampl_n': 25,
#     'lr': 1e-2,
#     'epoch_n': 10,
#     'batch': 2048,
#     'device': 'cpu',
#     'stop_pat': 4,
#     'lr_sched_pat': 1
# }
    
# histories[exp_name] = {
#     'hyps': hyps
# }

# BOOK_DIRNAME = '/kaggle/input/my-private-datasets/CVBooks/rus/'
# BOOK_LANGUAGE='russian'
# corpus_paragraphs = []
# for _, _, filenames in os.walk(BOOK_DIRNAME):
#     for filename in filenames:
#         #print(os.path.join(BOOK_DIRNAME, filename))
#         with open(os.path.join(BOOK_DIRNAME, filename)) as file:
#             corpus_paragraphs.extend([line.strip() for line in file])
            
# corpus_paragraphs = list(filter(None, corpus_paragraphs)) #remove empty strings
# print(len(corpus_paragraphs))
# corpus_paragraphs[200:205]

# #Для текущего эксперимента склеиваем все параграфы в единый текст.
# plain_corpus = ' '.join(corpus_paragraphs)
# #plain_corpus[1000:2000]

# #convert plain corpus to corpus of blocks with padding
# plain_corpus_tokenised = tokenize_corpus([plain_corpus], min_token_size=hyps['min_token_size'])[0]
# print(len(plain_corpus_tokenised))

# #partition into blocks
# block_len = hyps['MAX_SENTENCE_LEN'] - 2 * hyps['rwindow']

# corpus_blocks = []
# corpus_blocks.append(plain_corpus_tokenised[:hyps['rwindow']]) #only for padding

# for i in range(hyps['rwindow'], len(plain_corpus_tokenised) - block_len - hyps['rwindow']*2 + 1, block_len):
#     corpus_blocks.append(plain_corpus_tokenised[i:i+block_len])

# corpus_blocks.append(plain_corpus_tokenised[i:i+hyps['rwindow']]) #only for padding

# print(len(corpus_blocks) - 2)

# #add paddings
# padding = hyps['rwindow']
# for i in range(1, len(corpus_blocks) - 2):
#     corpus_blocks[i] = corpus_blocks[i-1][-padding:] + corpus_blocks[i] + corpus_blocks[i+1][:padding]
# corpus_blocks = corpus_blocks[1:-2]
# #print(corpus_blocks[10:14])

# #split to train/test
# random.shuffle(corpus_blocks)

# TRAIN_VAL_SPLIT = int(len(corpus_blocks) * 0.8)
# train_tokenized = corpus_blocks[:TRAIN_VAL_SPLIT]
# test_tokenized = corpus_blocks[TRAIN_VAL_SPLIT:]
# print("Обучающая выборка", len(train_tokenized))
# print("Тестовая выборка", len(test_tokenized))

# # строим словарь, используем для построения train_tokenized с удаленными паддингами
# vocabulary, word_doc_freq = build_vocabulary([block[padding:-padding] for block in train_tokenized], 
#                                              max_doc_freq=hyps['max_doc_freq'], 
#                                              min_count=hyps['min_count'], pad_word='<PAD>')
# histories[exp_name]['vocab_size'] = len(vocabulary)

# print("Размер словаря", len(vocabulary))
# print(list(vocabulary.items())[:10])

# # отображаем в номера токенов
# train_token_ids = texts_to_token_ids(train_tokenized, vocabulary)
# test_token_ids = texts_to_token_ids(test_tokenized, vocabulary)


# train_dataset = PaddedSequenceDataset(train_token_ids,
#                                       np.zeros(len(train_token_ids)),
#                                       out_len=hyps['MAX_SENTENCE_LEN'])
# test_dataset = PaddedSequenceDataset(test_token_ids,
#                                      np.zeros(len(test_token_ids)),
#                                      out_len=hyps['MAX_SENTENCE_LEN'])
# print(train_dataset[0], len(train_dataset[0][0]))

# ### Обучение
# trainer = SkipGramNegativeSamplingTrainer(len(vocabulary), hyps['emb_size'], hyps['MAX_SENTENCE_LEN'],
#                                           radius=hyps['rwindow'], negative_samples_n=hyps['neg_sampl_n'], with_padding=True)

# histories[exp_name]['train_history'], best_model = train_eval_loop(
#                                             trainer,
#                                             train_dataset,
#                                             test_dataset,
#                                             no_loss,
#                                             lr=hyps['lr'],
#                                             epoch_n=hyps['epoch_n'],
#                                             batch_size=hyps['batch'],
#                                             device=hyps['device'],
#                                             early_stopping_patience=hyps['stop_pat'],
#                                             max_batches_per_epoch_train=10000,
#                                             max_batches_per_epoch_val=len(test_dataset),
#                                             lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, 
#                                                                                                                        patience=hyps['lr_sched_pat'], 
#                                                                                                                        verbose=True),
#                                             no_calculate_accuracy = True)



# ### Исследуем характеристики полученных векторов
# embeddings = Embeddings(trainer.center_emb.weight.detach().cpu().numpy(), vocabulary)

# print('MOST_SIMILAR:')
# histories[exp_name]['most_similars'] = run_most_sumilars(embeddings.most_similar,WORDS_TO_SIMILAR)

# print('WORDS_ANALOGY:')
# print(WORDS_ANALOGY[0])
# print('\n'.join(map(str,embeddings.most_similar(**WORDS_ANALOGY[0], topk=10, with_mean=True))))

# print('\nWORDS_ON_PLOT:')
# words_to_plot = [w for w in WORDS_TO_PLOT if w in embeddings.word2id]
# test_vectors = embeddings.get_vectors(*words_to_plot)
# print(test_vectors.shape)

# histories[exp_name]['test_vectors'] = {word:vec for word,vec in zip(WORDS_TO_PLOT, test_vectors)}

# fig, ax = plt.subplots()
# fig.set_size_inches((10, 10))
# plot_vectors(np.array(list(histories[exp_name]['test_vectors'].values())), 
#              histories[exp_name]['test_vectors'].keys(), 
#              how='svd', ax=ax)

In [ ]:
# #hyperparameters
# exp_name = 'no_sentenses_rwindow=15_MAX_SEN_LEN=70_batch=2048'
# hyps = {
#     'min_token_size': 4,
#     'min_count': 2,
#     'max_doc_freq': 0.9, 

#     'MAX_SENTENCE_LEN': 35,
#     'emb_size': 100,
#     'rwindow': 12,
#     'padding': True,
    
#     'neg_sampl_n': 25,
#     'lr': 1e-2,
#     'epoch_n': 10,
#     'batch': 2048,
#     'device': 'cpu',
#     'stop_pat': 4,
#     'lr_sched_pat': 1
# }
    
# histories[exp_name] = {
#     'hyps': hyps
# }

# BOOK_DIRNAME = '/kaggle/input/my-private-datasets/CVBooks/rus/'
# BOOK_LANGUAGE='russian'
# corpus_paragraphs = []
# for _, _, filenames in os.walk(BOOK_DIRNAME):
#     for filename in filenames:
#         #print(os.path.join(BOOK_DIRNAME, filename))
#         with open(os.path.join(BOOK_DIRNAME, filename)) as file:
#             corpus_paragraphs.extend([line.strip() for line in file])
            
# corpus_paragraphs = list(filter(None, corpus_paragraphs)) #remove empty strings
# print(len(corpus_paragraphs))
# corpus_paragraphs[200:205]

# #Для текущего эксперимента склеиваем все параграфы в единый текст.
# plain_corpus = ' '.join(corpus_paragraphs)
# #plain_corpus[1000:2000]

# #convert plain corpus to corpus of blocks with padding
# plain_corpus_tokenised = tokenize_corpus([plain_corpus], min_token_size=hyps['min_token_size'])[0]
# print(len(plain_corpus_tokenised))

# #partition into blocks
# block_len = hyps['MAX_SENTENCE_LEN'] - 2 * hyps['rwindow']

# corpus_blocks = []
# corpus_blocks.append(plain_corpus_tokenised[:hyps['rwindow']]) #only for padding

# for i in range(hyps['rwindow'], len(plain_corpus_tokenised) - block_len - hyps['rwindow']*2 + 1, block_len):
#     corpus_blocks.append(plain_corpus_tokenised[i:i+block_len])

# corpus_blocks.append(plain_corpus_tokenised[i:i+hyps['rwindow']]) #only for padding

# print(len(corpus_blocks) - 2)

# #add paddings
# padding = hyps['rwindow']
# for i in range(1, len(corpus_blocks) - 2):
#     corpus_blocks[i] = corpus_blocks[i-1][-padding:] + corpus_blocks[i] + corpus_blocks[i+1][:padding]
# corpus_blocks = corpus_blocks[1:-2]
# #print(corpus_blocks[10:14])

# #split to train/test
# random.shuffle(corpus_blocks)

# TRAIN_VAL_SPLIT = int(len(corpus_blocks) * 0.8)
# train_tokenized = corpus_blocks[:TRAIN_VAL_SPLIT]
# test_tokenized = corpus_blocks[TRAIN_VAL_SPLIT:]
# print("Обучающая выборка", len(train_tokenized))
# print("Тестовая выборка", len(test_tokenized))

# # строим словарь, используем для построения train_tokenized с удаленными паддингами
# vocabulary, word_doc_freq = build_vocabulary([block[padding:-padding] for block in train_tokenized], 
#                                              max_doc_freq=hyps['max_doc_freq'], 
#                                              min_count=hyps['min_count'], pad_word='<PAD>')
# histories[exp_name]['vocab_size'] = len(vocabulary)

# print("Размер словаря", len(vocabulary))
# print(list(vocabulary.items())[:10])

# # отображаем в номера токенов
# train_token_ids = texts_to_token_ids(train_tokenized, vocabulary)
# test_token_ids = texts_to_token_ids(test_tokenized, vocabulary)


# train_dataset = PaddedSequenceDataset(train_token_ids,
#                                       np.zeros(len(train_token_ids)),
#                                       out_len=hyps['MAX_SENTENCE_LEN'])
# test_dataset = PaddedSequenceDataset(test_token_ids,
#                                      np.zeros(len(test_token_ids)),
#                                      out_len=hyps['MAX_SENTENCE_LEN'])
# print(train_dataset[0], len(train_dataset[0][0]))

# ### Обучение
# trainer = SkipGramNegativeSamplingTrainer(len(vocabulary), hyps['emb_size'], hyps['MAX_SENTENCE_LEN'],
#                                           radius=hyps['rwindow'], negative_samples_n=hyps['neg_sampl_n'], with_padding=True)

# histories[exp_name]['train_history'], best_model = train_eval_loop(
#                                             trainer,
#                                             train_dataset,
#                                             test_dataset,
#                                             no_loss,
#                                             lr=hyps['lr'],
#                                             epoch_n=hyps['epoch_n'],
#                                             batch_size=hyps['batch'],
#                                             device=hyps['device'],
#                                             early_stopping_patience=hyps['stop_pat'],
#                                             max_batches_per_epoch_train=10000,
#                                             max_batches_per_epoch_val=len(test_dataset),
#                                             lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, 
#                                                                                                                        patience=hyps['lr_sched_pat'], 
#                                                                                                                        verbose=True),
#                                             no_calculate_accuracy = True)



# ### Исследуем характеристики полученных векторов
# embeddings = Embeddings(trainer.center_emb.weight.detach().cpu().numpy(), vocabulary)

# print('MOST_SIMILAR:')
# histories[exp_name]['most_similars'] = run_most_sumilars(embeddings.most_similar,WORDS_TO_SIMILAR)

# print('WORDS_ANALOGY:')
# print(WORDS_ANALOGY[0])
# print('\n'.join(map(str,embeddings.most_similar(**WORDS_ANALOGY[0], topk=10, with_mean=True))))

# print('\nWORDS_ON_PLOT:')
# words_to_plot = [w for w in WORDS_TO_PLOT if w in embeddings.word2id]
# test_vectors = embeddings.get_vectors(*words_to_plot)
# print(test_vectors.shape)

# histories[exp_name]['test_vectors'] = {word:vec for word,vec in zip(WORDS_TO_PLOT, test_vectors)}

# fig, ax = plt.subplots()
# fig.set_size_inches((10, 10))
# plot_vectors(np.array(list(histories[exp_name]['test_vectors'].values())), 
#              histories[exp_name]['test_vectors'].keys(), 
#              how='svd', ax=ax)

### Эксперимент - изменить токенизацию, например, разобравшись с библиотекой SpaCy и подключив лемматизацию и POS-теггинг, чтобы строить эмбеддинги не для словоформ, а для лемм (например, chicked_NOUN)

In [ ]:
# def get_first_lemmas_by_word(word, lemmas2id):
#         found = False
#         for lemma in lemmas2id:
#             if word + '_' in lemma:
#                 found = True
#                 break
#         return lemma if found else None
# def get_lemmas_by_words(list_words, lemmas2id):
#     lemmas = [get_first_lemmas_by_word(word, lemmas2id) for word in list_words]
#     return list(filter(lambda lem: lem != None, lemmas))

In [ ]:
# BOOK_DIRNAME = '/kaggle/input/my-private-datasets/CVBooks/rus/'
# BOOK_LANGUAGE='russian'
# corpus_paragraphs = []
# for _, _, filenames in os.walk(BOOK_DIRNAME):
#     for filename in filenames:
#         #print(os.path.join(BOOK_DIRNAME, filename))
#         with open(os.path.join(BOOK_DIRNAME, filename)) as file:
#             corpus_paragraphs.extend([line.strip() for line in file])
            
# corpus_paragraphs = list(filter(None, corpus_paragraphs)) #remove empty strings
# print(len(corpus_paragraphs))
# corpus_paragraphs[200:205]

# #convert corpus of paragraphs to corpus of sentences
# corpus_sentences = []
# for paragraph in corpus_paragraphs:
#     corpus_sentences.extend(sent_tokenize(paragraph, language=BOOK_LANGUAGE))

# print(len(corpus_sentences))
# corpus_sentences[200:205]

# random.shuffle(corpus_sentences)
# corpus_sentences[:5]

# TRAIN_VAL_SPLIT = int(len(corpus_sentences) * 0.8)
# train_source = corpus_sentences[:TRAIN_VAL_SPLIT]
# test_source = corpus_sentences[TRAIN_VAL_SPLIT:]
# print("Обучающая выборка", len(train_source))
# print("Тестовая выборка", len(test_source))
# print()
# print('\n'.join(train_source[:5]))

In [ ]:
# #hyperparameters
# exp_name = 'lemmas_pos'
# hyps = {
#     'min_token_size': 3,
#     'min_count': 2,
#     'max_doc_freq': 0.9, 
#     'lem_with_pos': True,
#     'lem_remove_stopw': False,

#     'MAX_SENTENCE_LEN': 35,
#     'emb_size': 100,
#     'rwindow': 5,
    
#     'neg_sampl_n': 25,
#     'lr': 1e-2,
#     'epoch_n': 50,
#     'batch': 2048,
#     'device': 'cpu',
#     'stop_pat': 4,
#     'lr_sched_pat': 1
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# # токенизируем (с лемматизацией)
# train_tokenized = tokenize_corpus_verbose(train_source, tokenizer=tokenize_text_spacy_lemmatize, 
#                                           spacy_nlp=spacy_nlp, verbose_chunk=5000, 
#                                           min_token_size=hyps['min_token_size'], 
#                                           with_pos = hyps['lem_with_pos'], 
#                                           remove_stopwords = hyps['lem_remove_stopw'])
# test_tokenized = tokenize_corpus_verbose(test_source, tokenizer=tokenize_text_spacy_lemmatize, 
#                                           spacy_nlp=spacy_nlp, verbose_chunk=5000, 
#                                           min_token_size=hyps['min_token_size'], 
#                                           with_pos = hyps['lem_with_pos'], 
#                                           remove_stopwords = hyps['lem_remove_stopw'])

# print('\n'.join(' '.join(sent) for sent in train_tokenized[:10]))

# # строим словарь
# vocabulary, word_doc_freq = build_vocabulary(train_tokenized, 
#                                              max_doc_freq=hyps['max_doc_freq'], 
#                                              min_count=hyps['min_count'], pad_word='<PAD>')
# histories[exp_name]['vocab_size'] = len(vocabulary)

# print("Размер словаря", len(vocabulary))
# print(list(vocabulary.items())[:10])

# # отображаем в номера токенов
# train_token_ids = texts_to_token_ids(train_tokenized, vocabulary)
# test_token_ids = texts_to_token_ids(test_tokenized, vocabulary)

# # plt.hist([len(s) for s in train_token_ids], bins=20);
# # plt.title('Гистограмма длин предложений');

# print('MAX_SENTENCE_LEN = {}, Sentences counts: (<=) = {}, (>) = {}'.format(hyps['MAX_SENTENCE_LEN'],
#                                                 sum(np.array([len(s) for s in train_token_ids]) <= hyps['MAX_SENTENCE_LEN']),
#                                                 sum(np.array([len(s) for s in train_token_ids]) > hyps['MAX_SENTENCE_LEN'])))

# train_dataset = PaddedSequenceDataset(train_token_ids,
#                                       np.zeros(len(train_token_ids)),
#                                       out_len=hyps['MAX_SENTENCE_LEN'])
# test_dataset = PaddedSequenceDataset(test_token_ids,
#                                      np.zeros(len(test_token_ids)),
#                                      out_len=hyps['MAX_SENTENCE_LEN'])
# #print(train_dataset[0])

# ### Обучение
# trainer = SkipGramNegativeSamplingTrainer(len(vocabulary), hyps['emb_size'], hyps['MAX_SENTENCE_LEN'],
#                                           radius=hyps['rwindow'], negative_samples_n=hyps['neg_sampl_n'])

# histories[exp_name]['train_history'], best_model = train_eval_loop(
#                                             trainer,
#                                             train_dataset,
#                                             test_dataset,
#                                             no_loss,
#                                             lr=hyps['lr'],
#                                             epoch_n=hyps['epoch_n'],
#                                             batch_size=hyps['batch'],
#                                             device=hyps['device'],
#                                             early_stopping_patience=hyps['stop_pat'],
#                                             max_batches_per_epoch_train=10000,
#                                             max_batches_per_epoch_val=len(test_dataset),
#                                             lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, 
#                                                                                                                        patience=hyps['lr_sched_pat'], 
#                                                                                                                        verbose=True),
#                                             no_calculate_accuracy = True)



# ### Исследуем характеристики полученных векторов
# embeddings = Embeddings(trainer.center_emb.weight.detach().cpu().numpy(), vocabulary)

# print('MOST_SIMILAR:')

# if hyps['lem_with_pos']:
#     lemmas_to_similar = get_lemmas_by_words(WORDS_TO_SIMILAR, embeddings.word2id)
#     histories[exp_name]['most_similars'] = run_most_sumilars(embeddings.most_similar,lemmas_to_similar)
# else:
#     histories[exp_name]['most_similars'] = run_most_sumilars(embeddings.most_similar,WORDS_TO_SIMILAR)

# print('WORDS_ANALOGY:')
# print(WORDS_ANALOGY[0])

# if hyps['lem_with_pos']:
#     lemmas_analogy = []
#     for analogy_query in WORDS_ANALOGY:
#         lemmas_analogy.append(
#         {
#             'positive': get_lemmas_by_words(analogy_query['positive'], embeddings.word2id),
#             'negative': get_lemmas_by_words(analogy_query['negative'], embeddings.word2id)
#         })
#     print('\n'.join(map(str,embeddings.most_similar(**lemmas_analogy[0], topk=10, with_mean=True))))
# else:
#     print('\n'.join(map(str,embeddings.most_similar(**WORDS_ANALOGY[0], topk=10, with_mean=True))))

# print('\nWORDS_ON_PLOT:')

# if hyps['lem_with_pos']:
#     words_to_plot = get_lemmas_by_words(WORDS_TO_PLOT, embeddings.word2id)
# else:
#     words_to_plot = [w for w in WORDS_TO_PLOT if w in embeddings.word2id]
# test_vectors = embeddings.get_vectors(*words_to_plot)
# print(test_vectors.shape)

# histories[exp_name]['test_vectors'] = {word:vec for word,vec in zip(words_to_plot, test_vectors)}

# fig, ax = plt.subplots()
# fig.set_size_inches((10, 10))
# plot_vectors(np.array(list(histories[exp_name]['test_vectors'].values())), 
#              histories[exp_name]['test_vectors'].keys(), 
#              how='svd', ax=ax)

In [ ]:
# #hyperparameters
# exp_name = 'lemmas_nopos'
# hyps = {
#     'min_token_size': 3,
#     'min_count': 2,
#     'max_doc_freq': 0.9, 
#     'lem_with_pos': False,
#     'lem_remove_stopw': False,

#     'MAX_SENTENCE_LEN': 35,
#     'emb_size': 100,
#     'rwindow': 5,
    
#     'neg_sampl_n': 25,
#     'lr': 1e-2,
#     'epoch_n': 50,
#     'batch': 2048,
#     'device': 'cpu',
#     'stop_pat': 4,
#     'lr_sched_pat': 1
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# # токенизируем (с лемматизацией)
# train_tokenized = tokenize_corpus_verbose(train_source, tokenizer=tokenize_text_spacy_lemmatize, 
#                                           spacy_nlp=spacy_nlp, verbose_chunk=5000, 
#                                           min_token_size=hyps['min_token_size'], 
#                                           with_pos = hyps['lem_with_pos'], 
#                                           remove_stopwords = hyps['lem_remove_stopw'])
# test_tokenized = tokenize_corpus_verbose(test_source, tokenizer=tokenize_text_spacy_lemmatize, 
#                                           spacy_nlp=spacy_nlp, verbose_chunk=5000, 
#                                           min_token_size=hyps['min_token_size'], 
#                                           with_pos = hyps['lem_with_pos'], 
#                                           remove_stopwords = hyps['lem_remove_stopw'])

# print('\n'.join(' '.join(sent) for sent in train_tokenized[:10]))

# # строим словарь
# vocabulary, word_doc_freq = build_vocabulary(train_tokenized, 
#                                              max_doc_freq=hyps['max_doc_freq'], 
#                                              min_count=hyps['min_count'], pad_word='<PAD>')
# histories[exp_name]['vocab_size'] = len(vocabulary)

# print("Размер словаря", len(vocabulary))
# print(list(vocabulary.items())[:10])

# # отображаем в номера токенов
# train_token_ids = texts_to_token_ids(train_tokenized, vocabulary)
# test_token_ids = texts_to_token_ids(test_tokenized, vocabulary)

# # plt.hist([len(s) for s in train_token_ids], bins=20);
# # plt.title('Гистограмма длин предложений');

# print('MAX_SENTENCE_LEN = {}, Sentences counts: (<=) = {}, (>) = {}'.format(hyps['MAX_SENTENCE_LEN'],
#                                                 sum(np.array([len(s) for s in train_token_ids]) <= hyps['MAX_SENTENCE_LEN']),
#                                                 sum(np.array([len(s) for s in train_token_ids]) > hyps['MAX_SENTENCE_LEN'])))

# train_dataset = PaddedSequenceDataset(train_token_ids,
#                                       np.zeros(len(train_token_ids)),
#                                       out_len=hyps['MAX_SENTENCE_LEN'])
# test_dataset = PaddedSequenceDataset(test_token_ids,
#                                      np.zeros(len(test_token_ids)),
#                                      out_len=hyps['MAX_SENTENCE_LEN'])
# #print(train_dataset[0])

# ### Обучение
# trainer = SkipGramNegativeSamplingTrainer(len(vocabulary), hyps['emb_size'], hyps['MAX_SENTENCE_LEN'],
#                                           radius=hyps['rwindow'], negative_samples_n=hyps['neg_sampl_n'])

# histories[exp_name]['train_history'], best_model = train_eval_loop(
#                                             trainer,
#                                             train_dataset,
#                                             test_dataset,
#                                             no_loss,
#                                             lr=hyps['lr'],
#                                             epoch_n=hyps['epoch_n'],
#                                             batch_size=hyps['batch'],
#                                             device=hyps['device'],
#                                             early_stopping_patience=hyps['stop_pat'],
#                                             max_batches_per_epoch_train=10000,
#                                             max_batches_per_epoch_val=len(test_dataset),
#                                             lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, 
#                                                                                                                        patience=hyps['lr_sched_pat'], 
#                                                                                                                        verbose=True),
#                                             no_calculate_accuracy = True)



# ### Исследуем характеристики полученных векторов
# embeddings = Embeddings(trainer.center_emb.weight.detach().cpu().numpy(), vocabulary)

# print('MOST_SIMILAR:')

# if hyps['lem_with_pos']:
#     lemmas_to_similar = get_lemmas_by_words(WORDS_TO_SIMILAR, embeddings.word2id)
#     histories[exp_name]['most_similars'] = run_most_sumilars(embeddings.most_similar,lemmas_to_similar)
# else:
#     histories[exp_name]['most_similars'] = run_most_sumilars(embeddings.most_similar,WORDS_TO_SIMILAR)

# print('WORDS_ANALOGY:')
# print(WORDS_ANALOGY[0])

# if hyps['lem_with_pos']:
#     lemmas_analogy = []
#     for analogy_query in WORDS_ANALOGY:
#         lemmas_analogy.append(
#         {
#             'positive': get_lemmas_by_words(analogy_query['positive'], embeddings.word2id),
#             'negative': get_lemmas_by_words(analogy_query['negative'], embeddings.word2id)
#         })
#     print('\n'.join(map(str,embeddings.most_similar(**lemmas_analogy[0], topk=10, with_mean=True))))
# else:
#     print('\n'.join(map(str,embeddings.most_similar(**WORDS_ANALOGY[0], topk=10, with_mean=True))))

# print('\nWORDS_ON_PLOT:')

# if hyps['lem_with_pos']:
#     words_to_plot = get_lemmas_by_words(WORDS_TO_PLOT, embeddings.word2id)
# else:
#     words_to_plot = [w for w in WORDS_TO_PLOT if w in embeddings.word2id]
# test_vectors = embeddings.get_vectors(*words_to_plot)
# print(test_vectors.shape)

# histories[exp_name]['test_vectors'] = {word:vec for word,vec in zip(words_to_plot, test_vectors)}

# fig, ax = plt.subplots()
# fig.set_size_inches((10, 10))
# plot_vectors(np.array(list(histories[exp_name]['test_vectors'].values())), 
#              histories[exp_name]['test_vectors'].keys(), 
#              how='svd', ax=ax)

In [ ]:
# #hyperparameters
# exp_name = 'lemmas_pos_nostopwords'
# hyps = {
#     'min_token_size': 3,
#     'min_count': 2,
#     'max_doc_freq': 0.9, 
#     'lem_with_pos': True,
#     'lem_remove_stopw': True,

#     'MAX_SENTENCE_LEN': 35,
#     'emb_size': 100,
#     'rwindow': 5,
    
#     'neg_sampl_n': 25,
#     'lr': 1e-2,
#     'epoch_n': 50,
#     'batch': 2048,
#     'device': 'cpu',
#     'stop_pat': 4,
#     'lr_sched_pat': 1
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# # токенизируем (с лемматизацией)
# train_tokenized = tokenize_corpus_verbose(train_source, tokenizer=tokenize_text_spacy_lemmatize, 
#                                           spacy_nlp=spacy_nlp, verbose_chunk=5000, 
#                                           min_token_size=hyps['min_token_size'], 
#                                           with_pos = hyps['lem_with_pos'], 
#                                           remove_stopwords = hyps['lem_remove_stopw'])
# test_tokenized = tokenize_corpus_verbose(test_source, tokenizer=tokenize_text_spacy_lemmatize, 
#                                           spacy_nlp=spacy_nlp, verbose_chunk=5000, 
#                                           min_token_size=hyps['min_token_size'], 
#                                           with_pos = hyps['lem_with_pos'], 
#                                           remove_stopwords = hyps['lem_remove_stopw'])

# print('\n'.join(' '.join(sent) for sent in train_tokenized[:10]))

# # строим словарь
# vocabulary, word_doc_freq = build_vocabulary(train_tokenized, 
#                                              max_doc_freq=hyps['max_doc_freq'], 
#                                              min_count=hyps['min_count'], pad_word='<PAD>')
# histories[exp_name]['vocab_size'] = len(vocabulary)

# print("Размер словаря", len(vocabulary))
# print(list(vocabulary.items())[:10])

# # отображаем в номера токенов
# train_token_ids = texts_to_token_ids(train_tokenized, vocabulary)
# test_token_ids = texts_to_token_ids(test_tokenized, vocabulary)

# # plt.hist([len(s) for s in train_token_ids], bins=20);
# # plt.title('Гистограмма длин предложений');

# print('MAX_SENTENCE_LEN = {}, Sentences counts: (<=) = {}, (>) = {}'.format(hyps['MAX_SENTENCE_LEN'],
#                                                 sum(np.array([len(s) for s in train_token_ids]) <= hyps['MAX_SENTENCE_LEN']),
#                                                 sum(np.array([len(s) for s in train_token_ids]) > hyps['MAX_SENTENCE_LEN'])))

# train_dataset = PaddedSequenceDataset(train_token_ids,
#                                       np.zeros(len(train_token_ids)),
#                                       out_len=hyps['MAX_SENTENCE_LEN'])
# test_dataset = PaddedSequenceDataset(test_token_ids,
#                                      np.zeros(len(test_token_ids)),
#                                      out_len=hyps['MAX_SENTENCE_LEN'])
# #print(train_dataset[0])

# ### Обучение
# trainer = SkipGramNegativeSamplingTrainer(len(vocabulary), hyps['emb_size'], hyps['MAX_SENTENCE_LEN'],
#                                           radius=hyps['rwindow'], negative_samples_n=hyps['neg_sampl_n'])

# histories[exp_name]['train_history'], best_model = train_eval_loop(
#                                             trainer,
#                                             train_dataset,
#                                             test_dataset,
#                                             no_loss,
#                                             lr=hyps['lr'],
#                                             epoch_n=hyps['epoch_n'],
#                                             batch_size=hyps['batch'],
#                                             device=hyps['device'],
#                                             early_stopping_patience=hyps['stop_pat'],
#                                             max_batches_per_epoch_train=10000,
#                                             max_batches_per_epoch_val=len(test_dataset),
#                                             lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, 
#                                                                                                                        patience=hyps['lr_sched_pat'], 
#                                                                                                                        verbose=True),
#                                             no_calculate_accuracy = True)



# ### Исследуем характеристики полученных векторов
# embeddings = Embeddings(trainer.center_emb.weight.detach().cpu().numpy(), vocabulary)

# print('MOST_SIMILAR:')

# if hyps['lem_with_pos']:
#     lemmas_to_similar = get_lemmas_by_words(WORDS_TO_SIMILAR, embeddings.word2id)
#     histories[exp_name]['most_similars'] = run_most_sumilars(embeddings.most_similar,lemmas_to_similar)
# else:
#     histories[exp_name]['most_similars'] = run_most_sumilars(embeddings.most_similar,WORDS_TO_SIMILAR)

# print('WORDS_ANALOGY:')
# print(WORDS_ANALOGY[0])

# if hyps['lem_with_pos']:
#     lemmas_analogy = []
#     for analogy_query in WORDS_ANALOGY:
#         lemmas_analogy.append(
#         {
#             'positive': get_lemmas_by_words(analogy_query['positive'], embeddings.word2id),
#             'negative': get_lemmas_by_words(analogy_query['negative'], embeddings.word2id)
#         })
#     print('\n'.join(map(str,embeddings.most_similar(**lemmas_analogy[0], topk=10, with_mean=True))))
# else:
#     print('\n'.join(map(str,embeddings.most_similar(**WORDS_ANALOGY[0], topk=10, with_mean=True))))

# print('\nWORDS_ON_PLOT:')

# if hyps['lem_with_pos']:
#     words_to_plot = get_lemmas_by_words(WORDS_TO_PLOT, embeddings.word2id)
# else:
#     words_to_plot = [w for w in WORDS_TO_PLOT if w in embeddings.word2id]
# test_vectors = embeddings.get_vectors(*words_to_plot)
# print(test_vectors.shape)

# histories[exp_name]['test_vectors'] = {word:vec for word,vec in zip(words_to_plot, test_vectors)}

# fig, ax = plt.subplots()
# fig.set_size_inches((10, 10))
# plot_vectors(np.array(list(histories[exp_name]['test_vectors'].values())), 
#              histories[exp_name]['test_vectors'].keys(), 
#              how='svd', ax=ax)

In [ ]:
# #hyperparameters
# exp_name = 'lemmas_nopos_nostopwords'
# hyps = {
#     'min_token_size': 3,
#     'min_count': 2,
#     'max_doc_freq': 0.9, 
#     'lem_with_pos': False,
#     'lem_remove_stopw': True,

#     'MAX_SENTENCE_LEN': 35,
#     'emb_size': 100,
#     'rwindow': 5,
    
#     'neg_sampl_n': 25,
#     'lr': 1e-2,
#     'epoch_n': 50,
#     'batch': 2048,
#     'device': 'cpu',
#     'stop_pat': 4,
#     'lr_sched_pat': 1
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# # токенизируем (с лемматизацией)
# train_tokenized = tokenize_corpus_verbose(train_source, tokenizer=tokenize_text_spacy_lemmatize, 
#                                           spacy_nlp=spacy_nlp, verbose_chunk=5000, 
#                                           min_token_size=hyps['min_token_size'], 
#                                           with_pos = hyps['lem_with_pos'], 
#                                           remove_stopwords = hyps['lem_remove_stopw'])
# test_tokenized = tokenize_corpus_verbose(test_source, tokenizer=tokenize_text_spacy_lemmatize, 
#                                           spacy_nlp=spacy_nlp, verbose_chunk=5000, 
#                                           min_token_size=hyps['min_token_size'], 
#                                           with_pos = hyps['lem_with_pos'], 
#                                           remove_stopwords = hyps['lem_remove_stopw'])

# print('\n'.join(' '.join(sent) for sent in train_tokenized[:10]))

# # строим словарь
# vocabulary, word_doc_freq = build_vocabulary(train_tokenized, 
#                                              max_doc_freq=hyps['max_doc_freq'], 
#                                              min_count=hyps['min_count'], pad_word='<PAD>')
# histories[exp_name]['vocab_size'] = len(vocabulary)

# print("Размер словаря", len(vocabulary))
# print(list(vocabulary.items())[:10])

# # отображаем в номера токенов
# train_token_ids = texts_to_token_ids(train_tokenized, vocabulary)
# test_token_ids = texts_to_token_ids(test_tokenized, vocabulary)

# # plt.hist([len(s) for s in train_token_ids], bins=20);
# # plt.title('Гистограмма длин предложений');

# print('MAX_SENTENCE_LEN = {}, Sentences counts: (<=) = {}, (>) = {}'.format(hyps['MAX_SENTENCE_LEN'],
#                                                 sum(np.array([len(s) for s in train_token_ids]) <= hyps['MAX_SENTENCE_LEN']),
#                                                 sum(np.array([len(s) for s in train_token_ids]) > hyps['MAX_SENTENCE_LEN'])))

# train_dataset = PaddedSequenceDataset(train_token_ids,
#                                       np.zeros(len(train_token_ids)),
#                                       out_len=hyps['MAX_SENTENCE_LEN'])
# test_dataset = PaddedSequenceDataset(test_token_ids,
#                                      np.zeros(len(test_token_ids)),
#                                      out_len=hyps['MAX_SENTENCE_LEN'])
# #print(train_dataset[0])

# ### Обучение
# trainer = SkipGramNegativeSamplingTrainer(len(vocabulary), hyps['emb_size'], hyps['MAX_SENTENCE_LEN'],
#                                           radius=hyps['rwindow'], negative_samples_n=hyps['neg_sampl_n'])

# histories[exp_name]['train_history'], best_model = train_eval_loop(
#                                             trainer,
#                                             train_dataset,
#                                             test_dataset,
#                                             no_loss,
#                                             lr=hyps['lr'],
#                                             epoch_n=hyps['epoch_n'],
#                                             batch_size=hyps['batch'],
#                                             device=hyps['device'],
#                                             early_stopping_patience=hyps['stop_pat'],
#                                             max_batches_per_epoch_train=10000,
#                                             max_batches_per_epoch_val=len(test_dataset),
#                                             lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, 
#                                                                                                                        patience=hyps['lr_sched_pat'], 
#                                                                                                                        verbose=True),
#                                             no_calculate_accuracy = True)



# ### Исследуем характеристики полученных векторов
# embeddings = Embeddings(trainer.center_emb.weight.detach().cpu().numpy(), vocabulary)

# print('MOST_SIMILAR:')

# if hyps['lem_with_pos']:
#     lemmas_to_similar = get_lemmas_by_words(WORDS_TO_SIMILAR, embeddings.word2id)
#     histories[exp_name]['most_similars'] = run_most_sumilars(embeddings.most_similar,lemmas_to_similar)
# else:
#     histories[exp_name]['most_similars'] = run_most_sumilars(embeddings.most_similar,WORDS_TO_SIMILAR)

# print('WORDS_ANALOGY:')
# print(WORDS_ANALOGY[0])

# if hyps['lem_with_pos']:
#     lemmas_analogy = []
#     for analogy_query in WORDS_ANALOGY:
#         lemmas_analogy.append(
#         {
#             'positive': get_lemmas_by_words(analogy_query['positive'], embeddings.word2id),
#             'negative': get_lemmas_by_words(analogy_query['negative'], embeddings.word2id)
#         })
#     print('\n'.join(map(str,embeddings.most_similar(**lemmas_analogy[0], topk=10, with_mean=True))))
# else:
#     print('\n'.join(map(str,embeddings.most_similar(**WORDS_ANALOGY[0], topk=10, with_mean=True))))

# print('\nWORDS_ON_PLOT:')

# if hyps['lem_with_pos']:
#     words_to_plot = get_lemmas_by_words(WORDS_TO_PLOT, embeddings.word2id)
# else:
#     words_to_plot = [w for w in WORDS_TO_PLOT if w in embeddings.word2id]
# test_vectors = embeddings.get_vectors(*words_to_plot)
# print(test_vectors.shape)

# histories[exp_name]['test_vectors'] = {word:vec for word,vec in zip(words_to_plot, test_vectors)}

# fig, ax = plt.subplots()
# fig.set_size_inches((10, 10))
# plot_vectors(np.array(list(histories[exp_name]['test_vectors'].values())), 
#              histories[exp_name]['test_vectors'].keys(), 
#              how='svd', ax=ax)

In [ ]:
# #hyperparameters
# exp_name = 'lemmas_pos_min_token_size=2'
# hyps = {
#     'min_token_size': 2,
#     'min_count': 2,
#     'max_doc_freq': 0.9, 
#     'lem_with_pos': True,
#     'lem_remove_stopw': False,

#     'MAX_SENTENCE_LEN': 35,
#     'emb_size': 100,
#     'rwindow': 5,
    
#     'neg_sampl_n': 25,
#     'lr': 1e-2,
#     'epoch_n': 50,
#     'batch': 2048,
#     'device': 'cpu',
#     'stop_pat': 4,
#     'lr_sched_pat': 1
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# # токенизируем (с лемматизацией)
# train_tokenized = tokenize_corpus_verbose(train_source, tokenizer=tokenize_text_spacy_lemmatize, 
#                                           spacy_nlp=spacy_nlp, verbose_chunk=5000, 
#                                           min_token_size=hyps['min_token_size'], 
#                                           with_pos = hyps['lem_with_pos'], 
#                                           remove_stopwords = hyps['lem_remove_stopw'])
# test_tokenized = tokenize_corpus_verbose(test_source, tokenizer=tokenize_text_spacy_lemmatize, 
#                                           spacy_nlp=spacy_nlp, verbose_chunk=5000, 
#                                           min_token_size=hyps['min_token_size'], 
#                                           with_pos = hyps['lem_with_pos'], 
#                                           remove_stopwords = hyps['lem_remove_stopw'])

# print('\n'.join(' '.join(sent) for sent in train_tokenized[:10]))

# # строим словарь
# vocabulary, word_doc_freq = build_vocabulary(train_tokenized, 
#                                              max_doc_freq=hyps['max_doc_freq'], 
#                                              min_count=hyps['min_count'], pad_word='<PAD>')
# histories[exp_name]['vocab_size'] = len(vocabulary)

# print("Размер словаря", len(vocabulary))
# print(list(vocabulary.items())[:10])

# # отображаем в номера токенов
# train_token_ids = texts_to_token_ids(train_tokenized, vocabulary)
# test_token_ids = texts_to_token_ids(test_tokenized, vocabulary)

# # plt.hist([len(s) for s in train_token_ids], bins=20);
# # plt.title('Гистограмма длин предложений');

# print('MAX_SENTENCE_LEN = {}, Sentences counts: (<=) = {}, (>) = {}'.format(hyps['MAX_SENTENCE_LEN'],
#                                                 sum(np.array([len(s) for s in train_token_ids]) <= hyps['MAX_SENTENCE_LEN']),
#                                                 sum(np.array([len(s) for s in train_token_ids]) > hyps['MAX_SENTENCE_LEN'])))

# train_dataset = PaddedSequenceDataset(train_token_ids,
#                                       np.zeros(len(train_token_ids)),
#                                       out_len=hyps['MAX_SENTENCE_LEN'])
# test_dataset = PaddedSequenceDataset(test_token_ids,
#                                      np.zeros(len(test_token_ids)),
#                                      out_len=hyps['MAX_SENTENCE_LEN'])
# #print(train_dataset[0])

# ### Обучение
# trainer = SkipGramNegativeSamplingTrainer(len(vocabulary), hyps['emb_size'], hyps['MAX_SENTENCE_LEN'],
#                                           radius=hyps['rwindow'], negative_samples_n=hyps['neg_sampl_n'])

# histories[exp_name]['train_history'], best_model = train_eval_loop(
#                                             trainer,
#                                             train_dataset,
#                                             test_dataset,
#                                             no_loss,
#                                             lr=hyps['lr'],
#                                             epoch_n=hyps['epoch_n'],
#                                             batch_size=hyps['batch'],
#                                             device=hyps['device'],
#                                             early_stopping_patience=hyps['stop_pat'],
#                                             max_batches_per_epoch_train=10000,
#                                             max_batches_per_epoch_val=len(test_dataset),
#                                             lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, 
#                                                                                                                        patience=hyps['lr_sched_pat'], 
#                                                                                                                        verbose=True),
#                                             no_calculate_accuracy = True)



# ### Исследуем характеристики полученных векторов
# embeddings = Embeddings(trainer.center_emb.weight.detach().cpu().numpy(), vocabulary)

# print('MOST_SIMILAR:')

# if hyps['lem_with_pos']:
#     lemmas_to_similar = get_lemmas_by_words(WORDS_TO_SIMILAR, embeddings.word2id)
#     histories[exp_name]['most_similars'] = run_most_sumilars(embeddings.most_similar,lemmas_to_similar)
# else:
#     histories[exp_name]['most_similars'] = run_most_sumilars(embeddings.most_similar,WORDS_TO_SIMILAR)

# print('WORDS_ANALOGY:')
# print(WORDS_ANALOGY[0])

# if hyps['lem_with_pos']:
#     lemmas_analogy = []
#     for analogy_query in WORDS_ANALOGY:
#         lemmas_analogy.append(
#         {
#             'positive': get_lemmas_by_words(analogy_query['positive'], embeddings.word2id),
#             'negative': get_lemmas_by_words(analogy_query['negative'], embeddings.word2id)
#         })
#     print('\n'.join(map(str,embeddings.most_similar(**lemmas_analogy[0], topk=10, with_mean=True))))
# else:
#     print('\n'.join(map(str,embeddings.most_similar(**WORDS_ANALOGY[0], topk=10, with_mean=True))))

# print('\nWORDS_ON_PLOT:')

# if hyps['lem_with_pos']:
#     words_to_plot = get_lemmas_by_words(WORDS_TO_PLOT, embeddings.word2id)
# else:
#     words_to_plot = [w for w in WORDS_TO_PLOT if w in embeddings.word2id]
# test_vectors = embeddings.get_vectors(*words_to_plot)
# print(test_vectors.shape)

# histories[exp_name]['test_vectors'] = {word:vec for word,vec in zip(words_to_plot, test_vectors)}

# fig, ax = plt.subplots()
# fig.set_size_inches((10, 10))
# plot_vectors(np.array(list(histories[exp_name]['test_vectors'].values())), 
#              histories[exp_name]['test_vectors'].keys(), 
#              how='svd', ax=ax)

In [ ]:
# #hyperparameters
# exp_name = 'lemmas_pos_nostopwords_min_token_size=2'
# hyps = {
#     'min_token_size': 2,
#     'min_count': 2,
#     'max_doc_freq': 0.9, 
#     'lem_with_pos': True,
#     'lem_remove_stopw': True,

#     'MAX_SENTENCE_LEN': 35,
#     'emb_size': 100,
#     'rwindow': 5,
    
#     'neg_sampl_n': 25,
#     'lr': 1e-2,
#     'epoch_n': 50,
#     'batch': 2048,
#     'device': 'cpu',
#     'stop_pat': 4,
#     'lr_sched_pat': 1
# }

# histories[exp_name] = {
#     'hyps': hyps
# }

# # токенизируем (с лемматизацией)
# train_tokenized = tokenize_corpus_verbose(train_source, tokenizer=tokenize_text_spacy_lemmatize, 
#                                           spacy_nlp=spacy_nlp, verbose_chunk=5000, 
#                                           min_token_size=hyps['min_token_size'], 
#                                           with_pos = hyps['lem_with_pos'], 
#                                           remove_stopwords = hyps['lem_remove_stopw'])
# test_tokenized = tokenize_corpus_verbose(test_source, tokenizer=tokenize_text_spacy_lemmatize, 
#                                           spacy_nlp=spacy_nlp, verbose_chunk=5000, 
#                                           min_token_size=hyps['min_token_size'], 
#                                           with_pos = hyps['lem_with_pos'], 
#                                           remove_stopwords = hyps['lem_remove_stopw'])

# print('\n'.join(' '.join(sent) for sent in train_tokenized[:10]))

# # строим словарь
# vocabulary, word_doc_freq = build_vocabulary(train_tokenized, 
#                                              max_doc_freq=hyps['max_doc_freq'], 
#                                              min_count=hyps['min_count'], pad_word='<PAD>')
# histories[exp_name]['vocab_size'] = len(vocabulary)

# print("Размер словаря", len(vocabulary))
# print(list(vocabulary.items())[:10])

# # отображаем в номера токенов
# train_token_ids = texts_to_token_ids(train_tokenized, vocabulary)
# test_token_ids = texts_to_token_ids(test_tokenized, vocabulary)

# # plt.hist([len(s) for s in train_token_ids], bins=20);
# # plt.title('Гистограмма длин предложений');

# print('MAX_SENTENCE_LEN = {}, Sentences counts: (<=) = {}, (>) = {}'.format(hyps['MAX_SENTENCE_LEN'],
#                                                 sum(np.array([len(s) for s in train_token_ids]) <= hyps['MAX_SENTENCE_LEN']),
#                                                 sum(np.array([len(s) for s in train_token_ids]) > hyps['MAX_SENTENCE_LEN'])))

# train_dataset = PaddedSequenceDataset(train_token_ids,
#                                       np.zeros(len(train_token_ids)),
#                                       out_len=hyps['MAX_SENTENCE_LEN'])
# test_dataset = PaddedSequenceDataset(test_token_ids,
#                                      np.zeros(len(test_token_ids)),
#                                      out_len=hyps['MAX_SENTENCE_LEN'])
# #print(train_dataset[0])

# ### Обучение
# trainer = SkipGramNegativeSamplingTrainer(len(vocabulary), hyps['emb_size'], hyps['MAX_SENTENCE_LEN'],
#                                           radius=hyps['rwindow'], negative_samples_n=hyps['neg_sampl_n'])

# histories[exp_name]['train_history'], best_model = train_eval_loop(
#                                             trainer,
#                                             train_dataset,
#                                             test_dataset,
#                                             no_loss,
#                                             lr=hyps['lr'],
#                                             epoch_n=hyps['epoch_n'],
#                                             batch_size=hyps['batch'],
#                                             device=hyps['device'],
#                                             early_stopping_patience=hyps['stop_pat'],
#                                             max_batches_per_epoch_train=10000,
#                                             max_batches_per_epoch_val=len(test_dataset),
#                                             lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, 
#                                                                                                                        patience=hyps['lr_sched_pat'], 
#                                                                                                                        verbose=True),
#                                             no_calculate_accuracy = True)



# ### Исследуем характеристики полученных векторов
# embeddings = Embeddings(trainer.center_emb.weight.detach().cpu().numpy(), vocabulary)

# print('MOST_SIMILAR:')

# if hyps['lem_with_pos']:
#     lemmas_to_similar = get_lemmas_by_words(WORDS_TO_SIMILAR, embeddings.word2id)
#     histories[exp_name]['most_similars'] = run_most_sumilars(embeddings.most_similar,lemmas_to_similar)
# else:
#     histories[exp_name]['most_similars'] = run_most_sumilars(embeddings.most_similar,WORDS_TO_SIMILAR)

# print('WORDS_ANALOGY:')
# print(WORDS_ANALOGY[0])

# if hyps['lem_with_pos']:
#     lemmas_analogy = []
#     for analogy_query in WORDS_ANALOGY:
#         lemmas_analogy.append(
#         {
#             'positive': get_lemmas_by_words(analogy_query['positive'], embeddings.word2id),
#             'negative': get_lemmas_by_words(analogy_query['negative'], embeddings.word2id)
#         })
#     print('\n'.join(map(str,embeddings.most_similar(**lemmas_analogy[0], topk=10, with_mean=True))))
# else:
#     print('\n'.join(map(str,embeddings.most_similar(**WORDS_ANALOGY[0], topk=10, with_mean=True))))

# print('\nWORDS_ON_PLOT:')

# if hyps['lem_with_pos']:
#     words_to_plot = get_lemmas_by_words(WORDS_TO_PLOT, embeddings.word2id)
# else:
#     words_to_plot = [w for w in WORDS_TO_PLOT if w in embeddings.word2id]
# test_vectors = embeddings.get_vectors(*words_to_plot)
# print(test_vectors.shape)

# histories[exp_name]['test_vectors'] = {word:vec for word,vec in zip(words_to_plot, test_vectors)}

# fig, ax = plt.subplots()
# fig.set_size_inches((10, 10))
# plot_vectors(np.array(list(histories[exp_name]['test_vectors'].values())), 
#              histories[exp_name]['test_vectors'].keys(), 
#              how='svd', ax=ax)

### Эксперимент - реализовать FastText и сравнить, как отличаются списки похожих документов, получаемых с помощью Word2Vec и FastText

In [9]:
def get_sym_ngrams(token, size=(3, 6)):
    ngram = [token]
    leng = len(token)
    if leng > size[0]:
        for l in range(size[0], size[1]+1):
            ngram.extend([token[i:i+l] for i in range(0, leng - l + 1)])
    return ngram

def convert_corpus_to_sym_ngramms(corpus_tokenized, size=(3, 6)):
    #convert corpus_tokenized to corpus_sym_ngramms
        return [sum([get_sym_ngrams(token, size=size) for token in sent], []) for sent in corpus_tokenized]
    
def get_ngrams_idx(token, vocabulary_ngram, size=(3, 6)):
    if token in vocabulary_ngram:
        ngrams_idx = [vocabulary_ngram[token]]
    else:
        ngrams_idx = []
    leng = len(token)
    if leng > size[0]:
        for l in range(size[0], size[1]+1):
            ngrams_idx.extend([vocabulary_ngram[token[i:i+l]] for i in range(0, leng - l + 1) if token[i:i+l] in vocabulary_ngram])
    return ngrams_idx if ngrams_idx != [] else [0]

def get_corresp_voc_vocngram(vocabulary, vocabulary_ngram, size=(3, 6)):
    corresp = []
    for word, word_idx in vocabulary.items():
        corresp.append(get_ngrams_idx(word,vocabulary_ngram, size=size))
    corresp[0] = [0]
    return corresp

def convert_vocab_ngrams(corresp_vocngram, center_ngram_emb):
    center_embeddings = center_ngram_emb(torch.LongTensor(corresp_vocngram)[:,1:])  # VocabSize x MaxNgrams x MaxEmbSize
    center_embeddings = (center_embeddings.sum(-2)/center_embeddings.count_nonzero(-2)).nan_to_num() # VocabSize x MaxEmbSize
    return center_embeddings.data.detach().numpy()


class PaddedSequenceNgramsDataset(Dataset):
    def __init__(self, texts, corresp_vocngram, targets, sent_len=100, pad_value=0):
        self.texts = texts
        self.corresp_vocngram = corresp_vocngram
        self.targets = targets
        self.sent_len = sent_len
        self.max_ngrams_count = len(corresp_vocngram[0])
        self.pad_value = pad_value

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        txt = self.texts[item]

        txt = ensure_length(txt, self.sent_len, self.pad_value)
        txt = np.array([corresp_vocngram[idx] for idx in txt])
        txt = torch.tensor(txt, dtype=torch.long)

        target = torch.tensor(self.targets[item], dtype=torch.long)

        return txt, target

In [10]:
class SkipGramNegativeSamplingTrainer(nn.Module):
    #класс модифицирован для FastText
    def __init__(self, vocab_size, vocab_ngram_size, emb_size, sentence_len, radius=5, negative_samples_n=5):
        super().__init__()
        self.vocab_size = vocab_size
        self.vocab_ngram_size = vocab_ngram_size
        self.negative_samples_n = negative_samples_n

        self.center_ngram_emb = nn.Embedding(self.vocab_ngram_size, emb_size, padding_idx=0)
        self.center_ngram_emb.weight.data.uniform_(-1.0 / emb_size, 1.0 / emb_size)
        self.center_ngram_emb.weight.data[0] = 0

        self.context_emb = nn.Embedding(self.vocab_size, emb_size, padding_idx=0)
        self.context_emb.weight.data.uniform_(-1.0 / emb_size, 1.0 / emb_size)
        self.context_emb.weight.data[0] = 0

        self.positive_sim_mask = make_diag_mask(sentence_len, radius)
    
    
    def forward(self, sentences):
        """sentences - Batch x MaxSentLength - идентификаторы токенов"""
        batch_size = sentences.shape[0]

        #получает на вход LongTensor с idx (т.е. индексами токенов), возвращает тензор + 1 измерения
        #в котором индексы заменены на соответсвующие им embedding', равные среднему из всех ембеддингов,
        #входящих в токен n-gramm (это описание сразу 2-х строк ниже), среднее считается без учета 0
        center_embeddings = self.center_ngram_emb(sentences[:,:,1:])  # Batch x MaxSentLength x MaxNgrams x MaxEmbSize
        center_embeddings = (center_embeddings.sum(-2)/center_embeddings.count_nonzero(-2)).nan_to_num() # Batch x MaxSentLength x MaxEmbSize
        
        ### оценить сходство с настоящими соседними словами
        
        #получает на вход LongTensor с idx (т.е. индексами токенов), возвращает тензор + 1 измерения
        #в котором индексы заменены на соответсвующие им embedding'и, (это контекстные слова)
        #дополнительно транспонируем для целей последующего тензорного (матричного) умножения 
        #Итого(для batch=1): мы получаем тензор предложения фиксированной длины, где каждое слово 
        #заменено на embedding из контекстных слов, все отсутсвующие слова (нет в словаре или 
        #закончилось реальное предложение), заменяются на embedding из 0
        positive_context_embs = self.context_emb(sentences[:,:,0]).permute(0, 2, 1)  # Batch x EmbSize x MaxSentLength
        
        #перемножение тензоров, по сути скалярное произведение эмбеддингов, 
        #Важно отметить, что изначально я проедполагал, что эта операция равносильна нахождению косинусных расстояний, 
        #т.к. на основе анализа итоговых эмбеддингов, сделал неверный вывод, что длина каждого из векторов уже здесь = 1 
        #(т.е. они сразу нормализуются в пределах каждого embedding (например внутри класса torch.nn.Embedding), 
        #но это не так, нормализация происходит уже после полного обучения модели, через передачу весов в конструктор 
        #созданного вручную класса Embedding)
        #Итого(для batch=1): мы получаем матрицу MaxSentLength x MaxSentLength, скалярных произведений, 
        #между векторами каждого центрального слова и каждого контекстного слова (значения [-inf; inf])
        positive_sims = torch.bmm(center_embeddings, positive_context_embs)  # Batch x MaxSentLength x MaxSentLength
        
        #преобразуем в "условные вероятности" через взятие сигмоиды, т.е. получаем как бы 
        #"условные вероятности" встретить пары слов вместе, по факту для каждой пары, скалярное произведение, 
        #обернутое в сигмоиду и как следствие в диапазон значений (0; 1)
        positive_probs = torch.sigmoid(positive_sims)
        
        ### увеличить оценку вероятности встретить эти пары слов вместе
        
        #переводим тензор self.positive_sim_mask на тот же девайс, на котором positive_sims
        positive_mask = self.positive_sim_mask.to(positive_sims.device)
        
        #.expand_as - Expand this tensor to the same size as other. 
        #self.expand_as(other) is equivalent to self.expand(other.size())
        #positive_probs * positive_mask - мы оставляем только позиции пересечения центральных слов в контекстными,
        #все остальные позиции зануляются
        #подсчитываем бинарную кросс энтропию вычисленных "условных вероятностей" (сигмоид) и целевых = 1 для всех
        #пересечений центральных и контекстных слов, все остальные позиции в обоих матрицах = 0 
        #Примечание: т.к. по умолчанию BCEloss в реализации torch высчитывает итоговое значение как 'mean', 
        #а не 'sum' из всех полученных, то количество 0 так же влияет на итоговое значение, имеет ли это какой 
        #то эффект, и измениться ли что то, если выставить reduction='sum', не очевидно и нужно проверять на практике
        #Примечание: для всех позиций, которые занулены, их эмбеддинги соответсвуют эмбеддинг-вектору с idx=0, для
        #для которого мы при создании мы указали паддинг nn.Embedding(..., padding_idx=0), это означает, что эти веса
        #фиксированы, и не подлежат изменению через градиентных шаг
        #
        #Итого: важно понимать, что если бы оптимизировали только данную loss функцию, без отрицательных примеров, 
        #которые идут ниже, то, все сводилось бы к тому, что минимальное значение loss было бы, если бы мы все 
        #вектора (и центральных слов и контекстных) устремили бы в бесконечность, в одном направлении (например всем
        #их весам присвоили бы значение inf или любые подобные варианты)
        
        positive_loss = F.binary_cross_entropy(positive_probs * positive_mask,
                                               positive_mask.expand_as(positive_probs))

        
        ### выбрать случайные "отрицательные" слова
        negative_words = torch.randint(1, self.vocab_size,
                                       size=(batch_size, self.negative_samples_n),
                                       device=sentences.device)  # Batch x NegSamplesN
        negative_context_embs = self.context_emb(negative_words).permute(0, 2, 1)  # Batch x EmbSize x NegSamplesN
        negative_sims = torch.bmm(center_embeddings, negative_context_embs)  # Batch x MaxSentLength x NegSamplesN
        
        ### уменьшить оценку вероятность встретить эти пары слов вместе
        #Важно отметить, что BCEWithLogitsLoss расвносильна последовательному применению Sigmoid -> BCELoss
        #но в реализации torch она является более численно стабильной, чем раздельное применение
        #Итого: здесь все целевые (target) значения = 0, и если бы мы минимизировали только эту loss функцию, то минимальное
        #ее значение было бы, если бы мы устремили все вектора центральных слов в бесконечность одного направления, 
        #а вектора контекстных слов в бесконечность противоположного направления
        negative_loss = F.binary_cross_entropy_with_logits(negative_sims,
                                                           negative_sims.new_zeros(negative_sims.shape))
        return positive_loss + negative_loss


def no_loss(pred, target):
    """Фиктивная функция потерь - когда модель сама считает функцию потерь"""
    return pred

In [11]:
BOOK_DIRNAME = '/kaggle/input/my-private-datasets/CVBooks/rus/'
BOOK_LANGUAGE='russian'
corpus_paragraphs = []
for _, _, filenames in os.walk(BOOK_DIRNAME):
    for filename in filenames:
        #print(os.path.join(BOOK_DIRNAME, filename))
        with open(os.path.join(BOOK_DIRNAME, filename)) as file:
            corpus_paragraphs.extend([line.strip() for line in file])
            
corpus_paragraphs = list(filter(None, corpus_paragraphs)) #remove empty strings
print(len(corpus_paragraphs))
corpus_paragraphs[200:205]

#convert corpus of paragraphs to corpus of sentences
corpus_sentences = []
for paragraph in corpus_paragraphs:
    corpus_sentences.extend(sent_tokenize(paragraph, language=BOOK_LANGUAGE))

print(len(corpus_sentences))
corpus_sentences[200:205]

random.shuffle(corpus_sentences)
corpus_sentences[:5]

TRAIN_VAL_SPLIT = int(len(corpus_sentences) * 0.8)
train_source = corpus_sentences[:TRAIN_VAL_SPLIT]
test_source = corpus_sentences[TRAIN_VAL_SPLIT:]
print("Обучающая выборка", len(train_source))
print("Тестовая выборка", len(test_source))
print()
print('\n'.join(train_source[:5]))

In [12]:
#hyperparameters
exp_name = 'fasttext'
hyps = {
    'min_token_size': 3,
    'min_count': 2,
    'max_doc_freq': 0.9,
    
    'ngrams_size': (3, 6),
    'min_count_ngram': 5,
    'max_doc_freq_ngram': 0.9,

    'MAX_SENTENCE_LEN': 35,
    'emb_size': 100,
    'rwindow': 5,
    
    'neg_sampl_n': 25,
    'lr': 1e-2,
    'epoch_n': 30,
    'batch': 2048,
    'device': 'cpu',
    'stop_pat': 4,
    'lr_sched_pat': 1
}
    
histories[exp_name] = {
    'hyps': hyps
}

# токенизируем 
train_tokenized = tokenize_corpus(train_source, min_token_size=hyps['min_token_size'])
test_tokenized = tokenize_corpus(test_source, min_token_size=hyps['min_token_size'])
print('\n'.join(' '.join(sent) for sent in train_tokenized[:10]))

# строим словарь (основной словарь, только токены, без N-грамм)
vocabulary, word_doc_freq = build_vocabulary(train_tokenized, 
                                             max_doc_freq=hyps['max_doc_freq'], 
                                             min_count=hyps['min_count'], pad_word='<PAD>')
histories[exp_name]['vocab_size'] = len(vocabulary)

print("Размер словаря", len(vocabulary))
print(list(vocabulary.items())[:10])

# конвертируем корпус из токенов, в корпус их н-грамм символов, заданной размерности
train_tokenized_ngrams = convert_corpus_to_sym_ngramms(train_tokenized, size=hyps['ngrams_size'])
test_tokenized_ngrams = convert_corpus_to_sym_ngramms(test_tokenized, size=hyps['ngrams_size'])

# строим словарь (основной словарь, только токены, без N-грамм)
vocabulary_ngram, word_doc_freq_ngram = build_vocabulary(train_tokenized_ngrams, 
                                             max_doc_freq=hyps['max_doc_freq_ngram'], 
                                             min_count=hyps['min_count_ngram'], pad_word='<PAD>')
histories[exp_name]['vocab_ngram_size'] = len(vocabulary)

print("Размер словаря n-грамм", len(vocabulary_ngram))
print(list(vocabulary_ngram.items())[:10])


# отображаем в номера токенов
train_token_ids = texts_to_token_ids(train_tokenized, vocabulary)
test_token_ids = texts_to_token_ids(test_tokenized, vocabulary)


#составляем соответсвия для каждого idx токена из основного словаря, набору idx из словаря n-грамм  
corresp_vocngram = get_corresp_voc_vocngram(vocabulary, vocabulary_ngram, size=(3, 6))
max_ngrams_count = np.array(list(map(len, corresp_vocngram))).max()
corresp_vocngram = np.array([[word_id] + ngrams_idx + [0]*(max_ngrams_count - len(ngrams_idx)) for word_id, ngrams_idx in enumerate(corresp_vocngram)])

# plt.hist([len(s) for s in train_token_ids], bins=20);
# plt.title('Гистограмма длин предложений');

print('MAX_SENTENCE_LEN = {}, Sentences counts: (<=) = {}, (>) = {}'.format(hyps['MAX_SENTENCE_LEN'],
                                                sum(np.array([len(s) for s in train_token_ids]) <= hyps['MAX_SENTENCE_LEN']),
                                                sum(np.array([len(s) for s in train_token_ids]) > hyps['MAX_SENTENCE_LEN'])))


train_dataset = PaddedSequenceNgramsDataset(train_token_ids, corresp_vocngram,
                                            np.zeros(len(train_token_ids)),
                                            sent_len=hyps['MAX_SENTENCE_LEN'])
test_dataset = PaddedSequenceNgramsDataset(test_token_ids, corresp_vocngram,
                                           np.zeros(len(test_token_ids)),
                                           sent_len=hyps['MAX_SENTENCE_LEN'])

print(train_dataset[0])
print(train_dataset[0][0].shape)

### Обучение
trainer = SkipGramNegativeSamplingTrainer(len(vocabulary), len(vocabulary_ngram), 
                                          hyps['emb_size'], hyps['MAX_SENTENCE_LEN'],
                                          radius=hyps['rwindow'], negative_samples_n=hyps['neg_sampl_n'])

histories[exp_name]['train_history'], best_model = train_eval_loop(
                                            trainer,
                                            train_dataset,
                                            test_dataset,
                                            no_loss,
                                            lr=hyps['lr'],
                                            epoch_n=hyps['epoch_n'],
                                            batch_size=hyps['batch'],
                                            device=hyps['device'],
                                            early_stopping_patience=hyps['stop_pat'],
                                            max_batches_per_epoch_train=10000,
                                            max_batches_per_epoch_val=len(test_dataset),
                                            lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, 
                                                                                                                       patience=hyps['lr_sched_pat'], 
                                                                                                                       verbose=True),
                                            no_calculate_accuracy = True)

#что бы не переписывать функции ниже, конвертируем эмбеддинги n-грамм в эмбединги токенов 
#(каждый эмбединг токена, соответсвует среднему из эмбеддингов составляющих его н-грамм)
final_embeddings = convert_vocab_ngrams(corresp_vocngram, trainer.center_ngram_emb)

### Исследуем характеристики полученных векторов
embeddings = Embeddings(final_embeddings, vocabulary)

print('MOST_SIMILAR:')
histories[exp_name]['most_similars'] = run_most_sumilars(embeddings.most_similar,WORDS_TO_SIMILAR, topk=20)

print('WORDS_ANALOGY:')
print(WORDS_ANALOGY[0])
print('\n'.join(map(str,embeddings.most_similar(**WORDS_ANALOGY[0], topk=10, with_mean=True))))

print('\nWORDS_ON_PLOT:')
words_to_plot = [w for w in WORDS_TO_PLOT if w in embeddings.word2id]
test_vectors = embeddings.get_vectors(*words_to_plot)
print(test_vectors.shape)

histories[exp_name]['test_vectors'] = {word:vec for word,vec in zip(WORDS_TO_PLOT, test_vectors)}

fig, ax = plt.subplots()
fig.set_size_inches((10, 10))
plot_vectors(np.array(list(histories[exp_name]['test_vectors'].values())), 
             histories[exp_name]['test_vectors'].keys(), 
             how='svd', ax=ax)

In [84]:
def convert_vocab_ngrams(corresp_vocngram, center_ngram_emb):
    center_embeddings = center_ngram_emb(torch.LongTensor(corresp_vocngram)[:,1:])  # VocabSize x MaxNgrams x MaxEmbSize
    center_embeddings = center_embeddings.sum(-2) # VocabSize x MaxEmbSize
    return center_embeddings.data.detach().numpy()
                         
class SkipGramNegativeSamplingTrainer(nn.Module):
    #класс модифицирован для FastText (ВМЕСТО среднего, берется СУММА эмбеддингов н-грамм)
    def __init__(self, vocab_size, vocab_ngram_size, emb_size, sentence_len, radius=5, negative_samples_n=5):
        super().__init__()
        self.vocab_size = vocab_size
        self.vocab_ngram_size = vocab_ngram_size
        self.negative_samples_n = negative_samples_n

        self.center_ngram_emb = nn.Embedding(self.vocab_ngram_size, emb_size, padding_idx=0)
        self.center_ngram_emb.weight.data.uniform_(-1.0 / emb_size, 1.0 / emb_size)
        self.center_ngram_emb.weight.data[0] = 0

        self.context_emb = nn.Embedding(self.vocab_size, emb_size, padding_idx=0)
        self.context_emb.weight.data.uniform_(-1.0 / emb_size, 1.0 / emb_size)
        self.context_emb.weight.data[0] = 0

        self.positive_sim_mask = make_diag_mask(sentence_len, radius)
    
    
    def forward(self, sentences):
        """sentences - Batch x MaxSentLength - идентификаторы токенов"""
        batch_size = sentences.shape[0]

        #получает на вход LongTensor с idx (т.е. индексами токенов), возвращает тензор + 1 измерения
        #в котором индексы заменены на соответсвующие им embedding', равные сумме из всех ембеддингов,
        #входящих в токен n-gramm (это описание сразу 2-х строк ниже), среднее считается без учета 0
        center_embeddings = self.center_ngram_emb(sentences[:,:,1:])  # Batch x MaxSentLength x MaxNgrams x MaxEmbSize
        center_embeddings = center_embeddings.sum(-2) # Batch x MaxSentLength x MaxEmbSize
        
        ### оценить сходство с настоящими соседними словами
        
        #получает на вход LongTensor с idx (т.е. индексами токенов), возвращает тензор + 1 измерения
        #в котором индексы заменены на соответсвующие им embedding'и, (это контекстные слова)
        #дополнительно транспонируем для целей последующего тензорного (матричного) умножения 
        #Итого(для batch=1): мы получаем тензор предложения фиксированной длины, где каждое слово 
        #заменено на embedding из контекстных слов, все отсутсвующие слова (нет в словаре или 
        #закончилось реальное предложение), заменяются на embedding из 0
        positive_context_embs = self.context_emb(sentences[:,:,0]).permute(0, 2, 1)  # Batch x EmbSize x MaxSentLength
        
        #перемножение тензоров, по сути скалярное произведение эмбеддингов, 
        #Важно отметить, что изначально я проедполагал, что эта операция равносильна нахождению косинусных расстояний, 
        #т.к. на основе анализа итоговых эмбеддингов, сделал неверный вывод, что длина каждого из векторов уже здесь = 1 
        #(т.е. они сразу нормализуются в пределах каждого embedding (например внутри класса torch.nn.Embedding), 
        #но это не так, нормализация происходит уже после полного обучения модели, через передачу весов в конструктор 
        #созданного вручную класса Embedding)
        #Итого(для batch=1): мы получаем матрицу MaxSentLength x MaxSentLength, скалярных произведений, 
        #между векторами каждого центрального слова и каждого контекстного слова (значения [-inf; inf])
        positive_sims = torch.bmm(center_embeddings, positive_context_embs)  # Batch x MaxSentLength x MaxSentLength
        
        #преобразуем в "условные вероятности" через взятие сигмоиды, т.е. получаем как бы 
        #"условные вероятности" встретить пары слов вместе, по факту для каждой пары, скалярное произведение, 
        #обернутое в сигмоиду и как следствие в диапазон значений (0; 1)
        positive_probs = torch.sigmoid(positive_sims)
        
        ### увеличить оценку вероятности встретить эти пары слов вместе
        
        #переводим тензор self.positive_sim_mask на тот же девайс, на котором positive_sims
        positive_mask = self.positive_sim_mask.to(positive_sims.device)
        
        #.expand_as - Expand this tensor to the same size as other. 
        #self.expand_as(other) is equivalent to self.expand(other.size())
        #positive_probs * positive_mask - мы оставляем только позиции пересечения центральных слов в контекстными,
        #все остальные позиции зануляются
        #подсчитываем бинарную кросс энтропию вычисленных "условных вероятностей" (сигмоид) и целевых = 1 для всех
        #пересечений центральных и контекстных слов, все остальные позиции в обоих матрицах = 0 
        #Примечание: т.к. по умолчанию BCEloss в реализации torch высчитывает итоговое значение как 'mean', 
        #а не 'sum' из всех полученных, то количество 0 так же влияет на итоговое значение, имеет ли это какой 
        #то эффект, и измениться ли что то, если выставить reduction='sum', не очевидно и нужно проверять на практике
        #Примечание: для всех позиций, которые занулены, их эмбеддинги соответсвуют эмбеддинг-вектору с idx=0, для
        #для которого мы при создании мы указали паддинг nn.Embedding(..., padding_idx=0), это означает, что эти веса
        #фиксированы, и не подлежат изменению через градиентных шаг
        #
        #Итого: важно понимать, что если бы оптимизировали только данную loss функцию, без отрицательных примеров, 
        #которые идут ниже, то, все сводилось бы к тому, что минимальное значение loss было бы, если бы мы все 
        #вектора (и центральных слов и контекстных) устремили бы в бесконечность, в одном направлении (например всем
        #их весам присвоили бы значение inf или любые подобные варианты)
        
        positive_loss = F.binary_cross_entropy(positive_probs * positive_mask,
                                               positive_mask.expand_as(positive_probs))

        
        ### выбрать случайные "отрицательные" слова
        negative_words = torch.randint(1, self.vocab_size,
                                       size=(batch_size, self.negative_samples_n),
                                       device=sentences.device)  # Batch x NegSamplesN
        negative_context_embs = self.context_emb(negative_words).permute(0, 2, 1)  # Batch x EmbSize x NegSamplesN
        negative_sims = torch.bmm(center_embeddings, negative_context_embs)  # Batch x MaxSentLength x NegSamplesN
        
        ### уменьшить оценку вероятность встретить эти пары слов вместе
        #Важно отметить, что BCEWithLogitsLoss расвносильна последовательному применению Sigmoid -> BCELoss
        #но в реализации torch она является более численно стабильной, чем раздельное применение
        #Итого: здесь все целевые (target) значения = 0, и если бы мы минимизировали только эту loss функцию, то минимальное
        #ее значение было бы, если бы мы устремили все вектора центральных слов в бесконечность одного направления, 
        #а вектора контекстных слов в бесконечность противоположного направления
        negative_loss = F.binary_cross_entropy_with_logits(negative_sims,
                                                           negative_sims.new_zeros(negative_sims.shape))
        return positive_loss + negative_loss


def no_loss(pred, target):
    """Фиктивная функция потерь - когда модель сама считает функцию потерь"""
    return pred

In [12]:
#hyperparameters

#FastText (ВМЕСТО среднего, берется СУММА эмбеддингов н-грамм)
exp_name = 'fasttext_SUM'
hyps = {
    'min_token_size': 3,
    'min_count': 2,
    'max_doc_freq': 0.9,
    
    'ngrams_size': (3, 6),
    'min_count_ngram': 5,
    'max_doc_freq_ngram': 0.9,

    'MAX_SENTENCE_LEN': 35,
    'emb_size': 100,
    'rwindow': 5,
    
    'neg_sampl_n': 25,
    'lr': 1e-2,
    'epoch_n': 30,
    'batch': 2048,
    'device': 'cpu',
    'stop_pat': 4,
    'lr_sched_pat': 1
}
    
histories[exp_name] = {
    'hyps': hyps
}

# токенизируем 
train_tokenized = tokenize_corpus(train_source, min_token_size=hyps['min_token_size'])
test_tokenized = tokenize_corpus(test_source, min_token_size=hyps['min_token_size'])
print('\n'.join(' '.join(sent) for sent in train_tokenized[:10]))

# строим словарь (основной словарь, только токены, без N-грамм)
vocabulary, word_doc_freq = build_vocabulary(train_tokenized, 
                                             max_doc_freq=hyps['max_doc_freq'], 
                                             min_count=hyps['min_count'], pad_word='<PAD>')
histories[exp_name]['vocab_size'] = len(vocabulary)

print("Размер словаря", len(vocabulary))
print(list(vocabulary.items())[:10])

# конвертируем корпус из токенов, в корпус их н-грамм символов, заданной размерности
train_tokenized_ngrams = convert_corpus_to_sym_ngramms(train_tokenized, size=hyps['ngrams_size'])
test_tokenized_ngrams = convert_corpus_to_sym_ngramms(test_tokenized, size=hyps['ngrams_size'])

# строим словарь (основной словарь, только токены, без N-грамм)
vocabulary_ngram, word_doc_freq_ngram = build_vocabulary(train_tokenized_ngrams, 
                                             max_doc_freq=hyps['max_doc_freq_ngram'], 
                                             min_count=hyps['min_count_ngram'], pad_word='<PAD>')
histories[exp_name]['vocab_ngram_size'] = len(vocabulary)

print("Размер словаря n-грамм", len(vocabulary_ngram))
print(list(vocabulary_ngram.items())[:10])


# отображаем в номера токенов
train_token_ids = texts_to_token_ids(train_tokenized, vocabulary)
test_token_ids = texts_to_token_ids(test_tokenized, vocabulary)


#составляем соответсвия для каждого idx токена из основного словаря, набору idx из словаря n-грамм  
corresp_vocngram = get_corresp_voc_vocngram(vocabulary, vocabulary_ngram, size=(3, 6))
max_ngrams_count = np.array(list(map(len, corresp_vocngram))).max()
corresp_vocngram = np.array([[word_id] + ngrams_idx + [0]*(max_ngrams_count - len(ngrams_idx)) for word_id, ngrams_idx in enumerate(corresp_vocngram)])

# plt.hist([len(s) for s in train_token_ids], bins=20);
# plt.title('Гистограмма длин предложений');

print('MAX_SENTENCE_LEN = {}, Sentences counts: (<=) = {}, (>) = {}'.format(hyps['MAX_SENTENCE_LEN'],
                                                sum(np.array([len(s) for s in train_token_ids]) <= hyps['MAX_SENTENCE_LEN']),
                                                sum(np.array([len(s) for s in train_token_ids]) > hyps['MAX_SENTENCE_LEN'])))


train_dataset = PaddedSequenceNgramsDataset(train_token_ids, corresp_vocngram,
                                            np.zeros(len(train_token_ids)),
                                            sent_len=hyps['MAX_SENTENCE_LEN'])
test_dataset = PaddedSequenceNgramsDataset(test_token_ids, corresp_vocngram,
                                           np.zeros(len(test_token_ids)),
                                           sent_len=hyps['MAX_SENTENCE_LEN'])

print(train_dataset[0])
print(train_dataset[0][0].shape)

### Обучение
trainer = SkipGramNegativeSamplingTrainer(len(vocabulary), len(vocabulary_ngram), 
                                          hyps['emb_size'], hyps['MAX_SENTENCE_LEN'],
                                          radius=hyps['rwindow'], negative_samples_n=hyps['neg_sampl_n'])

histories[exp_name]['train_history'], best_model = train_eval_loop(
                                            trainer,
                                            train_dataset,
                                            test_dataset,
                                            no_loss,
                                            lr=hyps['lr'],
                                            epoch_n=hyps['epoch_n'],
                                            batch_size=hyps['batch'],
                                            device=hyps['device'],
                                            early_stopping_patience=hyps['stop_pat'],
                                            max_batches_per_epoch_train=10000,
                                            max_batches_per_epoch_val=len(test_dataset),
                                            lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, 
                                                                                                                       patience=hyps['lr_sched_pat'], 
                                                                                                                       verbose=True),
                                            no_calculate_accuracy = True)

#что бы не переписывать функции ниже, конвертируем эмбеддинги n-грамм в эмбединги токенов 
#(каждый эмбединг токена, соответсвует сумме из эмбеддингов составляющих его н-грамм)
final_embeddings = convert_vocab_ngrams(corresp_vocngram, trainer.center_ngram_emb)

### Исследуем характеристики полученных векторов
embeddings = Embeddings(final_embeddings, vocabulary)

print('MOST_SIMILAR:')
histories[exp_name]['most_similars'] = run_most_sumilars(embeddings.most_similar,WORDS_TO_SIMILAR, topk=20)

print('WORDS_ANALOGY:')
print(WORDS_ANALOGY[0])
print('\n'.join(map(str,embeddings.most_similar(**WORDS_ANALOGY[0], topk=10, with_mean=True))))

print('\nWORDS_ON_PLOT:')
words_to_plot = [w for w in WORDS_TO_PLOT if w in embeddings.word2id]
test_vectors = embeddings.get_vectors(*words_to_plot)
print(test_vectors.shape)

histories[exp_name]['test_vectors'] = {word:vec for word,vec in zip(WORDS_TO_PLOT, test_vectors)}

fig, ax = plt.subplots()
fig.set_size_inches((10, 10))
plot_vectors(np.array(list(histories[exp_name]['test_vectors'].values())), 
             histories[exp_name]['test_vectors'].keys(), 
             how='svd', ax=ax)

### Реализация FastText через gensim

In [35]:
import gensim

In [ ]:
exp_name = 'FastText_gensim'

hyps = {
    'min_token_size': 3,
    'min_count': 2,
    'emb_size': 100,
    'rwindow': 5,
    'ngrams_size': (3, 6)
}

histories[exp_name] = {
    'hyps': hyps
}

fasttext = gensim.models.FastText(sentences=train_tokenized, vector_size=hyps['emb_size'], 
                                  window=hyps['rwindow'], min_count=hyps['min_count'], 
                                  min_n=hyps['ngrams_size'][0], max_n=hyps['ngrams_size'][1],
                                  epochs=10)

histories[exp_name]['most_similars'] = run_most_sumilars(fasttext.wv.most_similar,WORDS_TO_SIMILAR, topn=20)
print(histories[exp_name]['most_similars'])

print(WORDS_ANALOGY[0])
fasttext.wv.most_similar(**WORDS_ANALOGY[0])

gensim_words = [w for w in WORDS_TO_PLOT if w in fasttext.wv.key_to_index]
gensim_vectors = np.stack([fasttext.wv[w] for w in gensim_words])
histories[exp_name]['test_vectors'] = {word:vec for word,vec in zip(gensim_words, gensim_vectors)}
fig, ax = plt.subplots()
fig.set_size_inches((10, 10))
plot_vectors(gensim_vectors, gensim_words, how='svd', ax=ax)

### Эксперимент - усложнить алгоритм оценки вероятности совместной встречаемости слов, например, заменив скалярное произведение на нейросеть с парой слоёв 

### Итоги экспериментов

In [ ]:
#show experiments min loss
train_history = {}
for exp_name, exp in histories.items():
    if 'train_history' in exp:
        train_history[exp_name] = exp['train_history']
show_experiments_stats(train_history, show_plots = False, only_BEST_MODEL_CALC = False)

In [ ]:
#Mutual distance (Cosine Similarity)

import seaborn as sns; sns.set_theme()

figsize = (14, 10)

print('Experiments:')
print('\n'.join(map(str,histories.keys())))
for word in WORDS_TO_SIMILAR[:3]:
    list_test_vectors = []
    for exp_name, exp_hist in histories.items():
        if ('lem_with_pos' in exp_hist['hyps']) and exp_hist['hyps']['lem_with_pos']:
            lemma = get_first_lemmas_by_word(word, exp_hist['test_vectors'])
            list_test_vectors.append(exp_hist['test_vectors'][lemma])
        else:
            list_test_vectors.append(exp_hist['test_vectors'][word])
        
    test_vectors = np.stack(list_test_vectors)
    similarity_corr = test_vectors @ test_vectors.T
    mask = np.zeros_like(similarity_corr)
    mask[np.triu_indices_from(mask)] = True
    with sns.axes_style("white"):
        f, ax = plt.subplots(figsize=figsize)
        ax.set_title(word)
        ax = sns.heatmap(similarity_corr, mask=mask, vmin=-1, vmax=1, annot=True, square=True, linewidths=.5, cmap="YlGnBu")


In [ ]:
#Mutual distance (SVD plots)

from sklearn.decomposition import TruncatedSVD
        
size_inches = (10, 10)
print('Experiments:')
print('\n'.join(map(str,histories.keys())))

for word in WORDS_TO_SIMILAR[:3]:
    print('WORD: {}'.format(word))
    list_test_vectors = []
    for exp_name, exp_hist in histories.items():
        if ('lem_with_pos' in exp_hist['hyps']) and exp_hist['hyps']['lem_with_pos']:
            lemma = get_first_lemmas_by_word(word, exp_hist['test_vectors'])
            list_test_vectors.append(exp_hist['test_vectors'][lemma])
        else:
            list_test_vectors.append(exp_hist['test_vectors'][word])

    if len(list_test_vectors) > 1:
        test_vectors = np.stack(list_test_vectors)
        print(TruncatedSVD().fit_transform(test_vectors))

        fig, ax = plt.subplots()
        fig.set_size_inches(size_inches)
        ax.set_title(word)
        plot_vectors(test_vectors, 
                     histories.keys(), 
                     how='svd', ax=ax, xy_lim=(-1., 1.))
    else:
        print('Only one experiment')

In [ ]:
#Mutual distance (SVD plots) in one plot

from sklearn.decomposition import TruncatedSVD
        
size_inches = (16, 16)

print('Experiments:')
print('\n'.join(map(str,histories.keys())))

list_test_vectors = []
list_test_vectors_names = []

for word in WORDS_TO_SIMILAR[:3]:
    for exp_name, exp_hist in histories.items():
        if ('lem_with_pos' in exp_hist['hyps']) and exp_hist['hyps']['lem_with_pos']:
            lemma = get_first_lemmas_by_word(word, exp_hist['test_vectors'])
            list_test_vectors.append(exp_hist['test_vectors'][lemma])
        else:
            list_test_vectors.append(exp_hist['test_vectors'][word])
        list_test_vectors_names.append('{}-{}'.format(word, exp_name))
        
test_vectors = np.stack(list_test_vectors)
    
fig, ax = plt.subplots()
fig.set_size_inches(size_inches)
plot_vectors(test_vectors, 
                 list_test_vectors_names, 
                 how='svd', ax=ax, xy_lim=(-1., 1.))

In [ ]:
#compare most_similars

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

list_compare_similars = []
for word in WORDS_TO_SIMILAR:
    list_similar = []
    for exp_name, exp_hist in histories.items():
        if ('lem_with_pos' in exp_hist['hyps']) and exp_hist['hyps']['lem_with_pos']:
            lemma = get_first_lemmas_by_word(word, exp_hist['test_vectors'])
            list_similar.append(exp_hist['most_similars'][lemma])
        else:
            list_similar.append(exp_hist['most_similars'][word])
    
    MultiIndex = pd.MultiIndex.from_product(([word], [i for i in range(10)]), names=["word", "n"])
    list_compare_similars.append(pd.DataFrame(list_similar,index=histories.keys(),columns=MultiIndex).T)
pd.concat(list_compare_similars)

In [ ]:
#Compare plots (SVD)
size_inches = (10, 10)
for exp_name, exp_hist in histories.items():
    fig, ax = plt.subplots()
    fig.set_size_inches(size_inches)
    ax.set_title('Experiment: {}'.format(exp_name))
    plot_vectors(np.array(list(exp_hist['test_vectors'].values())), 
                 exp_hist['test_vectors'].keys(), 
                 how='svd', ax=ax)

In [ ]:
#Compare plots (SVD) with xy_lim
size_inches = (10, 10)
for exp_name, exp_hist in histories.items():
    print('Experiment: {}'.format(exp_name))
    fig, ax = plt.subplots()
    fig.set_size_inches(size_inches)
    ax.set_title('Experiment: {}'.format(exp_name))
    plot_vectors(np.array(list(exp_hist['test_vectors'].values())), 
                 exp_hist['test_vectors'].keys(), 
                 how='svd', ax=ax, xy_lim=(-1., 1.))

In [ ]:
# #Compare plots (TSNE)
# size_inches = (10, 10)
# for exp_name, exp_hist in histories.items():
#     print('Experiment: {}'.format(exp_name))
#     fig, ax = plt.subplots()
#     fig.set_size_inches(size_inches)
#     plot_vectors(np.array(list(exp_hist['test_vectors'].values())), 
#                  exp_hist['test_vectors'].keys(), 
#                  how='tsne', ax=ax)